# 1. Import required modules

@ author: Shiwei Liu, Harvard University.

This script is written for analysis of mitotic cells during NE assembly. 

Images typically consist of a DNA channel, a ER/NE membrane marker channel, and another channel of interest which also can be used for cell segmentation, etc.

Below are the general walkthrough of the script:

1.User will define some common parameters for the data including where data is localized and the coordindates for the cell of interest if data needs to be cropped before analysis.

2.Masks for regions of interest will be generated by segmentation methods and addtional binary operations, which will be saved to the local directories.

3.Measurement for regions of interest will be performed, which will be exported to an excel table to the local directories.

In [1]:
# Put the AnalysisTool folder in the Documents folder;
# Next, replace the path within the "" with the path where AnalysisTool is saved; 
# (eg. "C:\Users\AAA\Documents\AnalysisTool\setup.py").
%run "C:\Users\Shiwei\Documents\AnalysisTool\setup.py"

# (eg. "C:\Users\AAA\Documents" for the python path.).
sys.path.append(r"C:\Users\Shiwei\Documents")

# Import the AnalysisTool packages below.
import AnalysisTool
from AnalysisTool import *
from AnalysisTool.mitosis_analysis import *
from AnalysisTool.mitosis_analysis_parameters import *

# (Optional) The PID below shows the program number of this running notebook (in Details in TaskManager); 
# which can be identified and close in TaskManager.
print(f'The program PID is {os.getpid()}')
print('The AnalysisTool is ready to use.')


c:\programdata\anaconda3\lib\site-packages\skimage\io\manage_plugins.py:23: UserWarning: Your installed pillow version is < 7.1.0. Several security issues (CVE-2020-11538, CVE-2020-10379, CVE-2020-10994, CVE-2020-10177) have been fixed in pillow 7.1.0 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


The program PID is 19516
The AnalysisTool is ready to use.


# 2. Define common parameters for data analysis

In [2]:
# Analysis parameters unique to each experiment

###########################################
# Data saved path: 
data_saved_mainfolder = r"D:\Analyzed_CellBio\Jing\3HeLa_NPC_assembly_on_laggard\20211103 HeLa GFP-Nup107 mCherry-STIM1 sirDNA Y27 plus MPS1i exp2"

data_saved_folder  = data_saved_mainfolder+ os.sep +r"exp1 1uM MPS1i"      
 
# If different cells are in single or multiple cell folder ('SINGLE' or 'MULTI')
# Data organization type:
data_folder_num = 'SINGLE'   
#data_folder_num = 'MULTI' 
##################################################
# Channel information:
# Number of channels used for the dataset:
num_of_ch=4
# Channel info and order:
#ch_dict = {"Transmitted":1,"STIM1":2,"Nup":3,"DNA":4}
ch_dict = {'Transmitted':1, "STIM1":3,"DNA":4, "NUP":2}        # Note it is 1561  2488!!!
# If manuanlly exclude Transmitted; 
# enable this only when the Transmitted has different timpoints compared to other channels;
# when Transmitted has same timpoints, disable this as the "ch_to_be_analyzed" below will exclude it;
# Channel indexes for channels to be analyzed; 
# Exclude 1 here for example since ch1 is transmitted and only has one z-plane.
ch_to_be_analyzed = [2,3,4]
# Define the channel index below using the info as above
# Channel index for DNA channel: 
# use negative dna exclusion or positive dna signal for dna segmentation 
dna_ch_index= 4
# Channel index for myo (or nup) channel:
# nup as myo
myo_ch_index=2
# Channel index for ER channel:  # Pseudo ER
er_ch_index=3
# Channel index for cytoplasm mask (e.g. sth that has a cytoplasmic cell background):
cell_ch_index =2

##################################################
# set this as True if want to analyze laggards for 5.1 below
_analyze_laggards_for_exp = True


##################################################
# Channel analysis selection:
# set if there is actual ER channel 
analyze_er = True
#analyze_er =False

# set actual myo channel as 'NUP' or 'MYO' or 'NONE'
_actual_myo_ch = 'NUP' 
#_actual_myo_ch = 'NE' 
#_actual_myo_ch = 'MYO' 
#_actual_myo_ch = 'NONE' 
#_actual_myo_ch = 'ACTIN' 
######################################
(data_folder_num,
num_of_ch,
ch_dict,
ch_to_be_analyzed,
dna_ch_index,
myo_ch_index,
er_ch_index,
cell_ch_index,
analyze_er,
_actual_myo_ch) = get_analysis_specific_parameters(data_saved_folder=data_saved_folder, 
                                     data_folder_num = data_folder_num,
                                     num_of_ch=num_of_ch,
                                     ch_dict = ch_dict ,
                                     ch_to_be_analyzed = ch_to_be_analyzed,
                                     dna_ch_index= dna_ch_index,
                                     myo_ch_index=myo_ch_index,
                                     er_ch_index=er_ch_index,
                                     cell_ch_index = cell_ch_index,
                                     analyze_er = analyze_er,
                                     _actual_myo_ch = _actual_myo_ch,
                                     overwrite_parameters=False)


Load parameters from the existing saved file.
Mannualy delete or change overwrite settings if want to make changes.


In [14]:
# Analysis parameters usually shared by most experiments
###########################################################################################
# Data type parameters: 
# Data analysis saving folder
data_analysis_folder = data_saved_folder

(overwrite_measurement,
timelapsed, _analyze_laggards,
_separate_inner_core_segment_measurement,
analyze_cell, num_of_segments_for_cell,
save_global_measurement, exclude_transmitted,
inverse_dna, extra_erosion_factor, 
_er_nup_overlap_factor,
num_of_segments, cytoplasm_ratio) = get_analysis_shared_parameters (_analyze_laggards = _analyze_laggards_for_exp,
                                    analyze_cell =True,
                                    exclude_transmitted=False,
                                    inverse_dna=False)

###########################################################################################
# objective-related parameters for ER channel: 
(non_ridge_segmentation, 
 enriched_membrane, 
 large_object_size, 
 ridge_sigma, 
 sm_fragment_size, 
 er_core_thickness) = get_er_ch_parameters (objective_type = '100x')

#############################################################
# Default settings that do not need to change 
#(corresponding to the _actual_myo_ch, or the second channel)
#==========================================================#
(analyze_myo, 
 myo_df_prefix, 
 std_ratio, 
 DNA_exclusion_for_NUP, 
 nup_core_thickness, 
 nup_rim_adjust_size,
 actin_ridge_sigma,
 actin_fragment_size) = get_myo_ch_parameters (_actual_myo_ch, 
                                               er_core_thickness=er_core_thickness, 
                                               ridge_sigma=ridge_sigma, sm_fragment_size=sm_fragment_size)

# modify channel index accordingly
if _actual_myo_ch == 'NONE':
    myo_ch_index = er_ch_index # use ER channel where myo_ch is used for cell mask segmentation
    
if not analyze_er:
    er_ch_index = dna_ch_index
    
 ################################################
# Get shared param dict from defined values above for the current measurement function kwargs

_shared_object_kwargs = {'save_measurements':True,
                  'save_masks':True,
                  'verbose':True,
                  ####### er analyze setting ########
                   'analyze_er':analyze_er,
                  'er_core_thickness':er_core_thickness,
                  'er_ridge_sigma':ridge_sigma,
                  'er_fragment_size':sm_fragment_size,
                  'non_ridge_segmentation':non_ridge_segmentation,
                  'enriched_membrane':enriched_membrane,
                  ####### nup/actin/myo etc analyze setting #####
                  'analyze_myo':analyze_myo,
                    '_actual_myo_ch':_actual_myo_ch,
                   'nup_core_thickness':nup_core_thickness,
                  'std_ratio':std_ratio,
                  'DNA_exclusion_for_myo':DNA_exclusion_for_NUP,
                  'actin_fragment_size':actin_fragment_size,
                  'actin_ridge_sigma':actin_ridge_sigma,
                  'myo_df_prefix':myo_df_prefix,
                  'nup_rim_adjust_size':nup_rim_adjust_size,
                 }

##############################################################
# Define measurment columns names for the experiment below
# If need to modify these columns, also need to change the pipeline analysis below for each measurement
(chromosome_measurement_columns, 
 cell_measurement_columns, 
 cell_actin_measurement_columns, 
 core_noncore_measurement_columns) = get_mitotic_analysis_measurement_columns (myo_df_prefix)


Get shared parameters.
Get ER (membrane) channel-related parameters for 100x objectives.
Get the second channel-related parameters for NUP signal.
Get the dataframe measurement columns.


# 3. Initiate Data localization

In [15]:
# Move all experiment files into a single folder "cells"
move_exps = True
keep_source= False
if move_exps and data_folder_num == 'SINGLE':
    consolidate_exp_to_single_fd (data_saved_folder, target_subfolder = "cells", keep_source=keep_source)
    
elif move_exps and data_folder_num == 'MULTI':
    relocate_extra_cells (data_saved_folder)
    
# Copy to prepare ims for getting cell info using a channel of interest (e.g., ch_to_save ="488" for using 488 channel)
make_cell_info_ims = True
if make_cell_info_ims:
    image_list_combined = generate_metamorph_data_list_each_cell (data_saved_folder, data_folder_num = data_folder_num)
    _ims_for_cell_info = get_ims_for_cell_info(image_list_combined, ch_to_save ="488", time_mode ='last', save_ims=True)
else:
    print('Use existing Cell Info excel.')

Experiments are already consolidated. Check folder to see.
Generating the folder for cell info ImageJ/FIJI use.


In [6]:
# break here before selecting cell of interest using ImageJ#
#break

In [24]:
# After xyz selection in ImageJ/FIJI, generate cell info file from the raw info file
if make_cell_info_ims:
    cell_info_raw_df = pd.read_csv(data_saved_folder+os.sep+"cell_info_ims"+os.sep+"cell_info_raw.csv")
    cell_info=create_cell_info_from_raw(data_saved_folder, cell_info_raw_df,generate_info_for_mask=True)
    mkfd_dna_masks (data_saved_folder, cell_info, mk_mask_fd=True)
    cell_info
else:
    print('Use existing Cell Info excel.')

Cell info excel created based on the cell_info_raw file. Edit this cell info if necessary.
DNA mask folder exists.


In [7]:
#image_list_combined[2]

In [25]:
# Localize the image data and cell info excel.

# Find all subfolders directly containing images.

image_list_combined = generate_metamorph_data_list_each_cell (data_saved_folder, data_folder_num = data_folder_num)


# Cell info loading from cell_info excel file:
cell_info_name = data_saved_folder+os.sep+"cell_info.xlsx"

if os.path.exists(cell_info_name) is False:
    create_cell_info_template (data_saved_folder, image_list_combined)

cell_info = pd.read_excel(data_saved_folder+os.sep+"cell_info.xlsx")
    
if len(cell_info) != len(image_list_combined):
    print ('Number of image folders does not match the number of cell objects in the cell info excel.')
if check_cell_info (cell_info) == False:
    print ("The cell info file is not valid.")

else:
    print ('Data and cell info are located.')

###################################################################################    


# Select data of interest
sel_image_list_combined = []
for _image_list, _cell_info in zip(image_list_combined,cell_info.iloc):
    if _cell_info['not_to_analyze'] !=1:
        sel_image_list_combined.append(_image_list)
        
        
        
# Retrieve time interval for timelapsed images:

t0 = os.path.getmtime(sel_image_list_combined[0][0])  # use mtime for file modified time especially for copied files
t1 = os.path.getmtime(sel_image_list_combined[0][1])

time_interval = round(t1-t0)

#####################################################################################
        
# Select cell info for data of interest        
sel_cell_info = cell_info[cell_info['not_to_analyze']!=1]

cell_info

Cell info has correct format.
Data and cell info are located.


,cell_id,cell_center_x,cell_center_y,cell_size,z_selected,dna_masks,not_to_analyze
0,cell1,494,802,437,3,dna_masks/cell1mask,NaN
1,cell2,519,793,463,1,dna_masks/cell2mask,NaN
2,cell3,477,711,535,3,dna_masks/cell3mask,NaN
3,cell4,624,844,493,1,dna_masks/cell4mask,NaN


In [26]:
sel_cell_info

,cell_id,cell_center_x,cell_center_y,cell_size,z_selected,dna_masks,not_to_analyze
0,cell1,494,802,437,3,dna_masks/cell1mask,NaN
1,cell2,519,793,463,1,dna_masks/cell2mask,NaN
2,cell3,477,711,535,3,dna_masks/cell3mask,NaN
3,cell4,624,844,493,1,dna_masks/cell4mask,NaN


# 4. Define current analysis method

In [43]:
# Define the chromosome object-based measurement function for this anaylysis here:
# Do not change unless 


def current_measurement_for_object(_object_id,_object_info,  
                                   ims_each_fov_3D,
                                   myo_ch_index=0,
                                   er_ch_index=0,
                                   ##################
                                   object_type ='main_chromosome',
                                   refined_midbox_dict = None,
                                   dna_mask_to_exclude = None,
                                   save_measurements = True,
                                   verbose =True,
                                   save_masks = True,
                                   ###############
                                   analyze_er = True,
                                   er_core_thickness =10,
                                   er_ridge_sigma =1,
                                   er_fragment_size=75,
                                   non_ridge_segmentation=False,
                                   enriched_membrane='ER',
                                   #################
                                   analyze_myo =  True, 
                                   _actual_myo_ch = 'MYO',
                                   nup_core_thickness = 10, 
                                   std_ratio = 3,
                                   nup_rim_adjust_size = 4,
                                   DNA_exclusion_for_myo = False, 
                                    actin_fragment_size = 100,
                                   actin_ridge_sigma =1,
                                    myo_df_prefix='ch2',
                                  ):
    
    
    # Generate a empty dict to store all the measurements for object:
    all_measurements_each_chr = {}
     ########################## ########################## ##########################
    ########################## for lagging chromosome #########################
    if object_type == 'lagging_chromosome':
        
        
        nup_recruit_mask_each_fov = []
        er_recruit_mask_each_fov = []
        dna_rim_mask_each_fov = []
        dna_mask_each_fov = []
        
        combined_nup_area = []
        combined_er_area = []
        combined_nup_dna_rim = []
        combined_er_dna_rim = []
        
        object_z_used = []
                
        # if has any objects of laggards
        if _object_info[0][0] is not None:
                 
            for sub_object_info in _object_info:
            
                ims_ch_xy = ims_each_fov_3D [:,sub_object_info[0],:,:]
                ########################## for DNA mask #########################
                # DNA mask
                dna_mask = sub_object_info[1]
                ########################## for Channel2(NUP or Actin) mask #########################
                # Nup mask
                print (f'Actual non-ER channel is {_actual_myo_ch}. Use the proper segmentation accordingly.')
                if analyze_myo:
                    if _actual_myo_ch == 'ACTIN' or _actual_myo_ch == 'ER':
                        nup_mask = er_membrane_mask_segmentation (ims_ch_xy,
                                                     er_ch_index =myo_ch_index, 
                                                     cell_center=cell_center, cell_size=cell_size, 
                                                              sm_fragment_size = actin_fragment_size,
                                             threshold_by_region = 'GLOBAL',
                                                             ridge_sigma = actin_ridge_sigma,) 

                        # Actin from DNA rim
                        dna_mask_nup_rim = dilation (dna_mask, disk(nup_core_thickness)) *  (dna_mask ==0)  
                    
                
                    elif _actual_myo_ch == 'NUP' or _actual_myo_ch == 'MYO' or _actual_myo_ch == 'NE':  
                        nup_mask = myo_enrichment_mask_segmentation (ims_ch_xy,
                                                         myo_ch_index =myo_ch_index,
                                                         dna_mask = dna_mask_to_exclude,
                                                 cell_center=cell_center, cell_size=cell_size,
                                                         threshold_by_region = 'GLOBAL',
                                                num_of_obj = 2, std_ratio =std_ratio, DNA_exclusion = DNA_exclusion_for_myo)
                        # analyze NUP (or MYO) enrich
                        # adjust the rim for NUP, which tends to be more close to the DNA
                        dna_mask_nup_rim_outer = dilation (dna_mask, disk(nup_core_thickness-nup_rim_adjust_size)) 
                        dna_mask_nup_rim_inner = erosion (dna_mask, disk(nup_rim_adjust_size)) 
                        dna_mask_nup_rim = dna_mask_nup_rim_outer * (dna_mask_nup_rim_inner ==0)
                
                else:
                    print('No NUP/actin analysis, return empty values.')
                    nup_mask = np.zeros(dna_mask.shape)
                    dna_mask_nup_rim = np.zeros(dna_mask.shape)
                    
                # enrich mask
                nup_enrich = dna_mask_nup_rim * nup_mask
                nup_recruit_mask_each_fov.append(nup_enrich)
                # enrich measurement
                combined_nup_area.append(np.sum(nup_enrich))
                combined_nup_dna_rim.append(np.sum(dna_mask_nup_rim))
                ########################## for Channel1 (ER) mask #########################
                # ER mask
                if analyze_er:
                    er_mask = er_membrane_mask_segmentation (ims_ch_xy,
                                                     er_ch_index =er_ch_index, 
                                                     cell_center=cell_center, cell_size=cell_size,
                                             threshold_by_region = 'GLOBAL', ridge_sigma = ridge_sigma,
                                                        sm_fragment_size=sm_fragment_size,
                                                            non_ridge_segmentation=non_ridge_segmentation,
                                                            enriched_membrane=enriched_membrane) 
                    # analyze ER enrich
                    dna_mask_er_rim = dilation (dna_mask, disk(er_core_thickness)) *  (dna_mask ==0)
                    
                # if no ER or not analyzin er, return empty mask
                else:
                    print('No ER analysis, return empty values.')
                    er_mask = np.zeros(dna_mask.shape)
                    dna_mask_er_rim =np.zeros(dna_mask.shape)
            
                # enrich mask
                er_enrich = dna_mask_er_rim * er_mask
                er_recruit_mask_each_fov.append(er_enrich)
                # enrich measurement
                combined_er_area.append(np.sum(er_enrich))
                combined_er_dna_rim.append(np.sum(dna_mask_er_rim))
                
                # DNA rim mask used for ER and DNA mask
                dna_rim_mask_each_fov.append(dna_mask_er_rim)
                dna_mask_each_fov.append(dna_mask)
                
                # py Z index starts from 1 
                object_z_used.append(sub_object_info[0]+1) # imageJ z-index
                
            ########################## Combining all laggards accordingly for measurement and masks #########################
            # Combine measurements from all sub-objects
            if analyze_myo == True:
                nup_recruit_area= sum(combined_nup_area)
                nup_dna_rim = sum(combined_nup_dna_rim)
                normalized_nup_area = nup_recruit_area/nup_dna_rim
            else:
                nup_recruit_area = np.nan
                nup_dna_rim = np.nan
                normalized_nup_area = np.nan
                
                
            er_recruit_area= sum(combined_er_area)
            er_dna_rim = sum(combined_er_dna_rim)   
            normalized_er_area = er_recruit_area/er_dna_rim     
            
            # save all masks
            if save_masks:
                # Combine individual myo and membrane recruitment masks to save:
                # Use z of each sub-object as label 
                if analyze_myo == True:
                    nup_recruit_mask_combined = np.zeros(dna_mask.shape)
                    for _mask_ind, _mask in enumerate(nup_recruit_mask_each_fov):
                        _mask = _mask * object_z_used[_mask_ind]
                        if np.sum(nup_recruit_mask_combined) == 0:
                            nup_recruit_mask_combined = _mask
                        else:
                            nup_recruit_mask_combined = nup_recruit_mask_combined + _mask
                    nup_recruit_mask_combined = color.label2rgb(nup_recruit_mask_combined, bg_label=0)
                
                dna_mask_combined = np.zeros(dna_mask.shape)
                for _mask_ind, _mask in enumerate(dna_mask_each_fov):
                    _mask = _mask * object_z_used[_mask_ind]
                    if np.sum(dna_mask_combined) == 0:
                        dna_mask_combined = _mask
                    else:
                        dna_mask_combined = dna_mask_combined + _mask
                dna_mask_combined = color.label2rgb(dna_mask_combined, bg_label=0)
                
                er_recruit_mask_combined = np.zeros(dna_mask.shape)
                for _mask_ind, _mask in enumerate(er_recruit_mask_each_fov):
                    _mask = _mask * object_z_used[_mask_ind]
                    if np.sum(er_recruit_mask_combined) == 0:
                        er_recruit_mask_combined = _mask
                    else:
                        er_recruit_mask_combined = er_recruit_mask_combined + _mask   
                er_recruit_mask_combined = color.label2rgb(er_recruit_mask_combined, bg_label=0)
                
                dna_rim_for_er_combined = np.zeros(dna_mask.shape)
                for _mask_ind, _mask in enumerate(dna_rim_mask_each_fov):
                    _mask = _mask * object_z_used[_mask_ind]
                    if np.sum(dna_rim_for_er_combined) == 0:
                        dna_rim_for_er_combined = _mask
                    else:
                        dna_rim_for_er_combined = dna_rim_for_er_combined + _mask      
                dna_rim_for_er_combined = color.label2rgb(dna_rim_for_er_combined, bg_label=0)
                
                if analyze_myo == True:
                    all_masks = [dna_mask_combined,dna_rim_for_er_combined,er_recruit_mask_combined,
                             nup_recruit_mask_combined]
                else:
                    all_masks = [dna_mask_combined,dna_rim_for_er_combined,er_recruit_mask_combined]
                    
                    
                    
        # if no valid objects    
        else:
            object_z_used = []
            normalized_nup_area = np.nan
            normalized_er_area =np.nan
            er_recruit_area=np.nan
            nup_recruit_area=np.nan
            er_dna_rim =np.nan
            nup_dna_rim =np.nan
            
            empty_mask=np.zeros((400, 400))
            print('No laggards were found. Skip.')
            if analyze_myo == True:
                all_masks = [empty_mask,empty_mask,empty_mask,empty_mask]
            else:
                all_masks = [empty_mask,empty_mask,empty_mask]


        # Append chr info:
        all_measurements_each_chr['object_type'] = object_type
        all_measurements_each_chr['object_id'] = _object_id + 3 # set laggards starts from 3 for now
        all_measurements_each_chr['object_z_slice'] =[object_z_used]
        all_measurements_each_chr['er_recruit_normalized_area'] =normalized_er_area
        all_measurements_each_chr[f'{myo_df_prefix}_recruit_normalized_area'] =normalized_nup_area
        
        all_measurements_each_chr['er_recruit_area'] =er_recruit_area
        all_measurements_each_chr[f'{myo_df_prefix}_recruit_area'] =nup_recruit_area
        all_measurements_each_chr['er_dna_rim'] =er_dna_rim
        all_measurements_each_chr[f'{myo_df_prefix}_dna_rim'] =nup_dna_rim
        
        combined_all_measurements_df = pd.DataFrame(all_measurements_each_chr)
        combined_all_measurements_df['inner_core_segment_number'] ="all_laggards"
        
          #return [combined_all_measurements_df, all_masks]


    ########################## #################### #########################        
    ########################## for main chromosomes #########################            
    elif object_type == 'main_chromosome':  
        
        nup_recruit_mask_each_fov = []
        er_recruit_mask_each_fov = []
        dna_rim_mask_each_fov = []
        dna_mask_each_fov = []
        
        combined_nup_area = []
        combined_er_area = []
        combined_nup_dna_rim = []
        combined_er_dna_rim = []
        
        object_z_used = []
        
        # if has any objects
        if _object_info[0][0] is not None:
            
            for sub_object_info in _object_info: # should have any one sub-object 
            
                ims_ch_xy = ims_each_fov_3D [:,sub_object_info[0],:,:]
                ########################## for DNA mask #########################
                # DNA mask
                dna_mask = sub_object_info[1]
                
                
                # py Z index starts from 0 
                object_z_used.append(sub_object_info[0]+1) # imageJ z-index
                
                # combined empty list to store all inner core segments into each row
                combined_all_measurements_df = []
                
                ########################## for Channel 1(ER) mask #########################
                # ER mask
                if analyze_er:
                    er_mask = er_membrane_mask_segmentation (ims_ch_xy,
                                                     er_ch_index =er_ch_index, 
                                                     cell_center=cell_center, cell_size=cell_size,
                                             threshold_by_region = 'GLOBAL',ridge_sigma=ridge_sigma,
                                                        sm_fragment_size=sm_fragment_size,
                                                            non_ridge_segmentation=non_ridge_segmentation,
                                                            enriched_membrane=enriched_membrane) 
                    # ER from DNA rim
                    dna_mask_er_rim = dilation (dna_mask, disk(er_core_thickness)) *  (dna_mask ==0)
                    
                else:
                    print('No ER analysis, return empty values.')
                    er_mask =np.zeros(dna_mask.shape)
                    dna_mask_er_rim =np.zeros(dna_mask.shape)
                
                 
                ########################## for Channel 2(Nup or actin) mask #########################
                # Actin mask as Nup mask
                print (f'Actual non-ER channel is {_actual_myo_ch}. Use the proper segmentation accordingly.')
                if analyze_myo: 
                    if _actual_myo_ch == 'ACTIN' or _actual_myo_ch == 'ER':
                        nup_mask = er_membrane_mask_segmentation (ims_ch_xy,
                                                     er_ch_index =myo_ch_index, 
                                                     cell_center=cell_center, cell_size=cell_size, 
                                                              sm_fragment_size = actin_fragment_size,
                                             threshold_by_region = 'GLOBAL',
                                                             ridge_sigma = actin_ridge_sigma,) 

                        # Actin from DNA rim
                        dna_mask_nup_rim = dilation (dna_mask, disk(nup_core_thickness)) *  (dna_mask ==0)  
                    
                
                    elif _actual_myo_ch == 'NUP' or _actual_myo_ch == 'MYO' or _actual_myo_ch == 'NE':  
                        nup_mask = myo_enrichment_mask_segmentation (ims_ch_xy,
                                                         myo_ch_index =myo_ch_index,
                                                         dna_mask = dna_mask_to_exclude,
                                                 cell_center=cell_center, cell_size=cell_size,
                                                         threshold_by_region = 'GLOBAL',
                                                num_of_obj = 2, std_ratio =std_ratio, DNA_exclusion = DNA_exclusion_for_myo)
                        # analyze NUP (or MYO) enrich
                        # adjust the rim for NUP, which tends to be more close to the DNA
                        dna_mask_nup_rim_outer = dilation (dna_mask, disk(nup_core_thickness-nup_rim_adjust_size)) 
                        dna_mask_nup_rim_inner = erosion (dna_mask, disk(nup_rim_adjust_size)) 
                        dna_mask_nup_rim = dna_mask_nup_rim_outer * (dna_mask_nup_rim_inner ==0)
                
                else:
                    print('No NUP/actin analysis, return empty values.')
                    nup_mask = np.zeros(dna_mask.shape)
                    dna_mask_nup_rim = np.zeros(dna_mask.shape)
                    
                
                for _midbox_segment_label, _midbox_segment_mask in refined_midbox_dict.items():
                    
                    # ER for inner core segment
                    _inner_core_mask_er = dna_mask_er_rim * _midbox_segment_mask
                    _inner_core_mask_er = dilation(_inner_core_mask_er, disk(3))
                    _inner_core_mask_er = erosion(_inner_core_mask_er, disk(3))
                    _inner_core_mask_er = remove_small_objects(_inner_core_mask_er, 100)
                    
                    er_enrich = _inner_core_mask_er * er_mask
                    
                    normalized_er_area = np.sum(er_enrich)/np.sum(_inner_core_mask_er)
                    
                    # Also append the measurement for the integrated total inner core:
                    combined_er_area.append(np.sum(er_enrich))
                    combined_er_dna_rim.append(np.sum(_inner_core_mask_er))
    
                    # Append the individual myo/nup and membrane recruitment mask:     
                    er_recruit_mask_each_fov.append(er_enrich)
                    dna_rim_mask_each_fov.append(_inner_core_mask_er)
                    dna_mask_each_fov.append(dna_mask)
                    
                    
                    if analyze_myo == True:
                        _inner_core_mask_nup = dna_mask_nup_rim * _midbox_segment_mask
                        _inner_core_mask_nup = dilation(_inner_core_mask_nup, disk(3))
                        _inner_core_mask_nup = erosion(_inner_core_mask_nup, disk(3))
                        _inner_core_mask_nup = remove_small_objects(_inner_core_mask_nup, 100)
                        
                        nup_enrich = _inner_core_mask_nup * nup_mask
                        normalized_nup_area = np.sum(nup_enrich)/np.sum(_inner_core_mask_nup)
                        
                        
                        # Also ppend the measurement for the integrated total inner core:
                        combined_nup_area.append(np.sum(nup_enrich))
                        combined_nup_dna_rim.append(np.sum(_inner_core_mask_nup))
                    
                        # Append the individual myo/nup and membrane recruitment mask:
                        nup_recruit_mask_each_fov.append(nup_enrich)
                        
                        # save masks and measurements
                        all_masks = [dna_mask, nup_mask, er_mask, _inner_core_mask_er, nup_enrich, er_enrich]
                    
                        all_measurements_each_chr['object_type'] = object_type
                        all_measurements_each_chr['object_id'] = _object_id +1
                        all_measurements_each_chr['object_z_slice'] =[object_z_used]
                        all_measurements_each_chr['er_recruit_normalized_area'] =normalized_er_area
                        all_measurements_each_chr[f'{myo_df_prefix}_recruit_normalized_area'] =normalized_nup_area
                    
                        all_measurements_each_chr['er_recruit_area'] =np.sum(er_enrich)
                        all_measurements_each_chr[f'{myo_df_prefix}_recruit_area'] =np.sum(nup_enrich)
                        all_measurements_each_chr['er_dna_rim'] =np.sum(_inner_core_mask_er)
                        all_measurements_each_chr[f'{myo_df_prefix}_dna_rim'] =np.sum(_inner_core_mask_nup)
                    
                        all_measurements_each_chr['inner_core_segment_number'] =_midbox_segment_label + 1
                    
                    else:
                        # save masks and measurements
                        all_masks = [dna_mask, er_mask, _inner_core_mask_er, er_enrich]
                    
                        all_measurements_each_chr['object_type'] = object_type
                        all_measurements_each_chr['object_id'] = _object_id +1
                        all_measurements_each_chr['object_z_slice'] =[object_z_used]
                        all_measurements_each_chr['er_recruit_normalized_area'] =normalized_er_area
                        all_measurements_each_chr[f'{myo_df_prefix}_recruit_normalized_area'] =np.nan
                    
                        all_measurements_each_chr['er_recruit_area'] =np.sum(er_enrich)
                        all_measurements_each_chr[f'{myo_df_prefix}_recruit_area'] =np.nan
                        all_measurements_each_chr['er_dna_rim'] =np.sum(_inner_core_mask_er)
                        all_measurements_each_chr[f'{myo_df_prefix}_dna_rim'] =np.nan
                    
                        all_measurements_each_chr['inner_core_segment_number'] =_midbox_segment_label + 1
                        
                    # append all inner core subobjects as well
                    all_measurements_each_chr_df = pd.DataFrame(all_measurements_each_chr)
            
                    if len(combined_all_measurements_df) ==0:
                        combined_all_measurements_df = all_measurements_each_chr_df
                    else:
                        combined_all_measurements_df = pd.concat([combined_all_measurements_df, all_measurements_each_chr_df])
                        
                
            # Combine all area measurementsfor the integrated inner core [result will be effectively eqaul to bin as 1]          
            er_recruit_area= sum(combined_er_area)
            er_dna_rim = sum(combined_er_dna_rim)
            normalized_er_area = er_recruit_area/ er_dna_rim 
            
            if analyze_myo == True:
                nup_recruit_area= sum(combined_nup_area)    
                nup_dna_rim = sum(combined_nup_dna_rim)
                normalized_nup_area = nup_recruit_area/nup_dna_rim
            
                all_measurements_each_chr['object_type'] = object_type
                all_measurements_each_chr['object_id'] = _object_id +1
                all_measurements_each_chr['object_z_slice'] =[object_z_used]
                all_measurements_each_chr['er_recruit_normalized_area'] =normalized_er_area
                all_measurements_each_chr[f'{myo_df_prefix}_recruit_normalized_area'] =normalized_nup_area
            
                all_measurements_each_chr['er_recruit_area'] =er_recruit_area
                all_measurements_each_chr[f'{myo_df_prefix}_recruit_area'] =nup_recruit_area
                all_measurements_each_chr['er_dna_rim'] =er_dna_rim
                all_measurements_each_chr[f'{myo_df_prefix}_dna_rim'] =nup_dna_rim
                
                all_measurements_each_chr['inner_core_segment_number'] = 'all_bins'
            
            else: 
                all_measurements_each_chr['object_type'] = object_type
                all_measurements_each_chr['object_id'] = _object_id +1
                all_measurements_each_chr['object_z_slice'] =[object_z_used]
                all_measurements_each_chr['er_recruit_normalized_area'] =normalized_er_area
                all_measurements_each_chr[f'{myo_df_prefix}_recruit_normalized_area'] =np.nan
            
                all_measurements_each_chr['er_recruit_area'] =er_recruit_area
                all_measurements_each_chr[f'{myo_df_prefix}_recruit_area'] =np.nan
                all_measurements_each_chr['er_dna_rim'] =er_dna_rim
                all_measurements_each_chr[f'{myo_df_prefix}_dna_rim'] =np.nan
                
                all_measurements_each_chr['inner_core_segment_number'] = 'all_bins'

            # append the integrated measurement
            all_measurements_each_chr_df = pd.DataFrame(all_measurements_each_chr)
            combined_all_measurements_df = pd.concat([combined_all_measurements_df, all_measurements_each_chr_df])
            
  
            if save_masks:
                # Combine individual masks to save:
                midbox_bin_number = len(refined_midbox_dict.keys())
                # Use bin index of each inner core bin as label 
                if analyze_myo == True:
                    nup_recruit_mask_combined = np.zeros(dna_mask.shape)
                    for _mask_ind, _mask in enumerate(nup_recruit_mask_each_fov):
                        # bin labels start from 1
                        _mask = _mask * (range(midbox_bin_number)[_mask_ind]+1)
                        if np.sum(nup_recruit_mask_combined) == 0:
                            nup_recruit_mask_combined = _mask
                        else:
                            nup_recruit_mask_combined = nup_recruit_mask_combined + _mask
                    nup_recruit_mask_combined = color.label2rgb(nup_recruit_mask_combined, bg_label=0)
                
                er_recruit_mask_combined = np.zeros(dna_mask.shape)
                for _mask_ind, _mask in enumerate(er_recruit_mask_each_fov):
                    # bin labels start from 1
                    _mask = _mask * (range(midbox_bin_number)[_mask_ind]+1)
                    if np.sum(er_recruit_mask_combined) == 0:
                        er_recruit_mask_combined = _mask
                    else:
                        er_recruit_mask_combined = er_recruit_mask_combined + _mask   
                er_recruit_mask_combined = color.label2rgb(er_recruit_mask_combined, bg_label=0)
                
                dna_rim_for_er_combined = np.zeros(dna_mask.shape)
                for _mask_ind, _mask in enumerate(dna_rim_mask_each_fov):
                    # bin labels start from 1
                    _mask = _mask * (range(midbox_bin_number)[_mask_ind]+1)
                    if np.sum(dna_rim_for_er_combined) == 0:
                        dna_rim_for_er_combined = _mask
                    else:
                        dna_rim_for_er_combined = dna_rim_for_er_combined + _mask
                dna_rim_for_er_combined = color.label2rgb(dna_rim_for_er_combined, bg_label=0)
                
                # use the first DNA mask (as all are same if multiple)
                dna_mask_combined = dna_mask_each_fov[0]
                 
                if analyze_myo == True:
                    all_masks = [dna_mask_combined, dna_rim_for_er_combined,
                             er_recruit_mask_combined,nup_recruit_mask_combined, er_mask, nup_mask]    
                else:
                    all_masks = [dna_mask_combined, dna_rim_for_er_combined,
                             er_recruit_mask_combined, er_mask]    

                
        # if no valid objects            
        else:
            normalized_nup_area = np.nan
            normalized_er_area =np.nan
            all_masks = []
            
            all_measurements_each_chr['object_type'] = object_type
            all_measurements_each_chr['object_id'] = _object_id +1
            all_measurements_each_chr['object_z_slice'] =[]
            all_measurements_each_chr['er_recruit_normalized_area'] =np.nan
            all_measurements_each_chr[f'{myo_df_prefix}_recruit_normalized_area'] =np.nan
            
            all_measurements_each_chr['er_recruit_area'] =np.nan
            all_measurements_each_chr[f'{myo_df_prefix}_recruit_area'] =np.nan
            all_measurements_each_chr['er_dna_rim'] =np.nan
            all_measurements_each_chr[f'{myo_df_prefix}_dna_rim'] =np.nan
            
            all_measurements_each_chr['inner_core_segment_number'] =[]
                    
            combined_all_measurements_df = pd.DataFrame(all_measurements_each_chr)

    ########################## report error #########################            
    else:
        print ('-- No such object type, exit.')
        
                    
    return [combined_all_measurements_df, all_masks]
            
            


# 5. Run image segmentation and analysis

## 5.1. Chromosome associated analysis

In [44]:
# Perform image segmentation and analysis for all data within the data_save_folder.
# Check, Edit, and Re-run all the boxes above if errors occur.
# Segmentation masks will overwrite everytime when it runs; 
# measurements will be saved as a new excel for the first run; for later runs, measurements will overwrite old ones.


for _index, _image_list in enumerate(sel_image_list_combined[1:2]):
    
    # Reading cell info and determine methods based on cell info:
    _cell_info = sel_cell_info.iloc[_index]
    
    cell_id = _cell_info['cell_id']
    
    if np.isnan(_cell_info['cell_center_x']) or np.isnan(_cell_info['cell_center_x']):
        cell_center = [750,750]
        threshold_by_region_method = 'GLOBAL'
    else:    
        cell_center = [round(_cell_info['cell_center_x']),round(_cell_info['cell_center_y'])]
        threshold_by_region_method = 'USER'

    if not np.isnan(_cell_info['cell_size']):
        cell_size = round(_cell_info['cell_size'])
    else:
        cell_size = 600
        
    if not np.isnan(_cell_info['z_selected']):
        z_selected = round(_cell_info['z_selected'])
        _use_pre_defined_Z = True
    else:
        z_selected = 1
        _use_pre_defined_Z = False
    
    
    if type(_cell_info['dna_masks']) is str:
        _mask_file_folder = data_saved_folder + os.sep+ _cell_info['dna_masks']
        if os.path.exists(_mask_file_folder):
            _use_saved_mask = True
        else:
            ("Cell info contains mask path, but no valid mask path/folder actually exists.")
            _mask_file_folder = ''
            _use_saved_mask = False
            
    else:
        _mask_file_folder = ''
        _use_saved_mask = False
        
#########################################################################################################################
#########################################################################################################################
    # Analyze each cell below.
    print (f'Starting analysis for cell_{cell_id}')
    print ('-----------------------------------------------------------------------------------------------------')
    print ('-----------------------------------------------------------------------------------------------------')
    import time
    start_time = time.time()
    
    #================================load images and massk for all timepoints========================================#
    
    
    # load metamorph images for each cell
    sorted_mm_data_array = load_metamorph_tifs_multi_z (_image_list, 
                                                       num_of_ch=num_of_ch, 
                                                       ch_to_be_analyzed = ch_to_be_analyzed,
                                                       timelapsed = timelapsed,
                                                       exclude_transmitted=exclude_transmitted)
    
        
    # generate mask list depending if the used saved mask files are correct.
    mask_filename_list = generate_mask_filename_list_for_timelapsed (len(sorted_mm_data_array),
                                                                     mask_file_folder=_mask_file_folder, 
                                                                          use_saved_mask = _use_saved_mask)
    
    # re-define channel index after channel exclusion
    new_dna_ch_index=ch_index_after_ch_exclusion (dna_ch_index,num_of_ch,ch_to_be_analyzed)
    new_myo_ch_index =ch_index_after_ch_exclusion (myo_ch_index,num_of_ch,ch_to_be_analyzed)
    new_er_ch_index =ch_index_after_ch_exclusion (er_ch_index,num_of_ch,ch_to_be_analyzed)
    new_cell_ch_index=ch_index_after_ch_exclusion (cell_ch_index,num_of_ch,ch_to_be_analyzed)
    
    
    # get object using 3D mask if DNA is stained positively
    if not inverse_dna: 
        # generate 3D mask for main chromosome mass
        _main_chromosome_mask = object_segmentation_timelapsed_3D (sorted_mm_data_array, object_ch_index = new_dna_ch_index, 
                            threshold_method = skimage.filters.thresholding.threshold_otsu, 
                            threshold_by_region = threshold_by_region_method, cell_center = cell_center, cell_size = cell_size,
                            large_object_size = large_object_size, object_type = "main_nuclei", midbox_mask = None)
    
        if _analyze_laggards:
            # use DNA at timepoint 1 to generate a rough midbox_mask for filtering lagging chromosomes
            
            max_dna_mask= np.max(_main_chromosome_mask[0], axis=0)
            max_dna_mask.astype(bool)
            max_dna_mask = remove_small_objects(max_dna_mask, large_object_size)
            rough_main_chromosome, num_of_rough_main_chromosome = ndi.label(max_dna_mask)
            
            if num_of_rough_main_chromosome==2:
                rough_midbox_max = generate_midbox_with_bins (max_dna_mask, num_of_bins=1,
                                          _verbose = True)[0]
            else:
                # use cell area to select laggards instead
                rough_midbox_max = object_segmentation_timelapsed_3D (sorted_mm_data_array, object_ch_index = new_cell_ch_index, 
                            threshold_method = skimage.filters.thresholding.threshold_otsu, 
                            threshold_by_region = threshold_by_region_method, cell_center = cell_center, cell_size = cell_size,
                            large_object_size = large_object_size, object_type = "main_nuclei", midbox_mask = None)
                
                rough_midbox_max =np.max(rough_midbox_max[0], axis=0)

            # generate 3D mask for lagging chromosomes
            _lagging_chromosome_mask = object_segmentation_timelapsed_3D (sorted_mm_data_array, object_ch_index = new_dna_ch_index, 
                            threshold_method = skimage.filters.thresholding.threshold_otsu, 
                            threshold_by_region = threshold_by_region_method, cell_center = cell_center, cell_size = cell_size,
                            large_object_size = large_object_size, object_type = "micronuclei", midbox_mask = rough_midbox_max)
    
    
    
        # generate chromosome objects to analyze
        if not _use_pre_defined_Z: # if use pre-specified Z for main chromosome
            main_chromosome_objects = simple_find_objects_timelapsed_3D (_main_chromosome_mask, combine_objects = False,
                                        search_radius = 75, object_area_change_ratio = 0.6,
                                        object_erosion_factor =3)
        if _analyze_laggards:
            lagging_chromosome_objects = simple_find_objects_timelapsed_3D (_lagging_chromosome_mask, combine_objects = True,
                                        search_radius = 75, object_area_change_ratio = 0.6,
                                        object_erosion_factor =3)
    
    else:
        print ("DNA should be stained for laggard analysis.")
        pass
    
    
    # set df for all timepoints (check the df columns definition above for details)
    _measurement_df_all_fov = pd.DataFrame(columns=chromosome_measurement_columns)
    
    #================================analyze each timepoint========================================#
     
    # _fov_id ~ _timepoint_index
    for _fov_id in range(len(sorted_mm_data_array)):
        
        
        print ('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
        print (f'-- Analyze TIMEPOINT {_fov_id + 1}')

        
        # set df for measurement (check the df columns definition above for details)
        _measurement_df_each_fov = pd.DataFrame(columns=chromosome_measurement_columns[2:-1])
        
        # set mask list to save
        _mask_each_fov = []
        
        # load image data for the corresponding timepoint
        ims_each_fov_3D = sorted_mm_data_array [_fov_id]
        
        
        # get pseudo or saved dna mask
        raw_image_shape = ims_each_fov_3D[0,0,:,:].shape
        saved_dna_mask = generate_masks_from_saved (mask_filename_list[_fov_id],raw_image_shape)
        
        saved_dna_mask = saved_dna_mask [int(cell_center[1]-cell_size/2):int(cell_center[1]+cell_size/2),  #crop x,y
                         int(cell_center[0]-cell_size/2):int(cell_center[0]+cell_size/2)]  
        
          
        # crop raw images if necessary
        if threshold_by_region_method == "USER":
            ims_each_fov_3D = ims_each_fov_3D[:,:,
                        int(cell_center[1]-cell_size/2):int(cell_center[1]+cell_size/2),    # channel crop x
                         int(cell_center[0]-cell_size/2):int(cell_center[0]+cell_size/2)]   # channel crop y
            
        ############################################################
        # Analyze the main chromosome masses:
        print ('-- Analyze the main chromosome masses.')
        
        # if use pre-specified Z for main chromosome
        if _use_pre_defined_Z: 
            # load xy for all channels for the pre-specified Z
            ims_ch_xy = ims_each_fov_3D [:,z_selected-1,:,:]
            
            # generate main chromosome mask 
            if not inverse_dna: 
                dna_mask = chromosome_mask_segmentation(ims_ch_xy,
                                                    dna_ch_index=new_dna_ch_index, 
                                                             cell_mask_ch_index =new_myo_ch_index, cell_center=cell_center,
                                                             cell_size=cell_size, 
                                                    small_object_th=large_object_size,
                                         threshold_by_region = "GLOBAL",inner_core_mode = False, 
                                                    chr_mask_from_saved = saved_dna_mask)
            else:            
                dna_mask = inverse_chromosome_mask_segmentation_by_line_profile(ims_ch_xy,
                                                    dna_ch_index=new_dna_ch_index, 
                                                             cell_center=cell_center,
                                                             cell_size=cell_size, 
                                                            extra_erosion_factor = extra_erosion_factor,
                                         threshold_by_region = "GLOBAL",inner_core_mode = False, 
                                                non_chr_low_ratio =0.09, chr_peak_prominence = 10,
                                                    chr_mask_from_saved = saved_dna_mask)
                
            # repeat the size filtering once with the same setting in case of unknown error occured in prior function
            dna_mask= dna_mask.astype(bool)
            dna_mask = remove_small_objects(dna_mask, large_object_size)
            
            # this generate or replaced the midbox based on the refined dna mask 
            refined_midbox_dict = generate_midbox_with_bins (dna_mask, num_of_bins=num_of_segments,_verbose = True)
            
            labeled_main_chromosome, num_of_main_chromosome = ndi.label(dna_mask)

            dna_mask_to_exclude=erosion(dna_mask, disk(10))
            
            if num_of_main_chromosome == 2:
                for _object_id, _main_chromosome_id in enumerate(range(num_of_main_chromosome)):
                    object_mask = labeled_main_chromosome == _main_chromosome_id+1
                    object_z = z_selected -1 # py z-index
                    object_info = [[object_z, object_mask]]
                    
                    
                    object_measurement = current_measurement_for_object (_object_id,object_info, 
                                                                         ims_each_fov_3D,
                                                                          myo_ch_index=new_myo_ch_index,
                                                                     er_ch_index=new_er_ch_index,
                                                                        object_type = 'main_chromosome',
                                                                        refined_midbox_dict = refined_midbox_dict,
                                                                        dna_mask_to_exclude =dna_mask_to_exclude,
                                                                        **_shared_object_kwargs) 
                    # append the result for each object
                    _measurement_df_each_fov = pd.concat([_measurement_df_each_fov, object_measurement[0]])
                    
                    # append unique masks
                    if _object_id == 0:
                        if analyze_myo == True:
                            for _mask in object_measurement[1][-2:]:
                                _mask_each_fov.append(_mask)
                            for _mask in object_measurement[1][:-2]:
                                _mask_each_fov.append(_mask)  
                        else:
                            for _mask in object_measurement[1][-1:]:
                                _mask_each_fov.append(_mask)
                            for _mask in object_measurement[1][:-1]:
                                _mask_each_fov.append(_mask) 
                            
                    else:
                        if analyze_myo == True:
                            for _mask in object_measurement[1][:-2]:
                                _mask_each_fov.append(_mask)
                        else:
                            for _mask in object_measurement[1][:-1]:
                                _mask_each_fov.append(_mask) 
                            
                    
             
        # if use objects identified by 'simple_find_objects_timelapsed_3D'
        else:
            main_chromosome_objects_each_fov = main_chromosome_objects[_fov_id]
            
            for _object_id, _object_info in enumerate(main_chromosome_objects_each_fov.values()):
                object_measurement = current_measurement_for_object (_object_id,_object_info, 
                                                                    ims_each_fov_3D,
                                                                     myo_ch_index=new_myo_ch_index,
                                                                     er_ch_index=new_er_ch_index,
                                                                    object_type = 'main_chromosome',
                                                                    refined_midbox_dict = refined_midbox_dict,
                                                                    dna_mask_to_exclude =dna_mask_to_exclude,
                                                                    **_shared_object_kwargs)
            # append the result for each object
            _measurement_df_each_fov = pd.concat([_measurement_df_each_fov, object_measurement[0]])
            
            # append unique masks
            if _object_id == 0:
                if analyze_myo == True:
                    for _mask in object_measurement[1][-2:]:
                        _mask_each_fov.append(_mask)
                    for _mask in object_measurement[1][:-2]:
                        _mask_each_fov.append(_mask)  
                else:
                    for _mask in object_measurement[1][-1:]:
                        _mask_each_fov.append(_mask)
                    for _mask in object_measurement[1][:-1]:
                        _mask_each_fov.append(_mask) 
                            
            else:
                if analyze_myo == True:
                    for _mask in object_measurement[1][:-2]:
                        _mask_each_fov.append(_mask)
                else:
                    for _mask in object_measurement[1][:-1]:
                        _mask_each_fov.append(_mask) 
                
        ############################################################   
        
        if _analyze_laggards:
            # Analyze the lagging chromosomes:
            print ('-- Analyze the lagging chromosomes.')
            lagging_chromosome_objects_each_fov = lagging_chromosome_objects[_fov_id]
        
            for _object_id,_object_info in enumerate(lagging_chromosome_objects_each_fov.values()):
     
                object_measurement = current_measurement_for_object (_object_id, _object_info,   #one object with multiple sub-objects
                                                                ims_each_fov_3D, 
                                                                myo_ch_index=new_myo_ch_index,
                                                                er_ch_index=new_er_ch_index,
                                                                 object_type = 'lagging_chromosome',
                                                                 refined_midbox_dict = refined_midbox_dict,
                                                                dna_mask_to_exclude =dna_mask_to_exclude,
                                                                **_shared_object_kwargs) 
        
                # append the result for each object
                _measurement_df_each_fov = pd.concat([_measurement_df_each_fov, object_measurement[0]])
            
                # append masks
                for _mask in object_measurement[1]:
                    _mask_each_fov.append(_mask)
            
        ############################################################
        # Save mask images
        mask_save_path =data_analysis_folder +os.sep+"chrom_segmentations"+os.sep+cell_id
        if not os.path.exists(mask_save_path):
            os.makedirs(mask_save_path)
            
        mask_savename = [] 
        if timelapsed == True:
            mask_name_prefix = 'time_'
        else:
            mask_name_prefix = 'fov_'
            
        mask_savename.append(mask_name_prefix + str(_fov_id+1) + "_er_mask.tif")
        if analyze_myo == True:
            mask_savename.append(mask_name_prefix + str(_fov_id+1) + f"_{myo_df_prefix}_mask.tif")
        
        mask_savename.append(mask_name_prefix + str(_fov_id+1) + "_main_chr_1_dna.tif")
        mask_savename.append(mask_name_prefix + str(_fov_id+1) + "_main_chr_1_core.tif")
        mask_savename.append(mask_name_prefix + str(_fov_id+1) + "_main_chr_1_er.tif")
        if analyze_myo == True:
            mask_savename.append(mask_name_prefix + str(_fov_id+1) + f"_main_chr_1_{myo_df_prefix}.tif")
        
        mask_savename.append(mask_name_prefix + str(_fov_id+1) + "_main_chr_2_dna.tif")
        mask_savename.append(mask_name_prefix + str(_fov_id+1) + "_main_chr_2_core.tif")
        mask_savename.append(mask_name_prefix + str(_fov_id+1) + "_main_chr_2_er.tif")
        if analyze_myo == True:
            mask_savename.append(mask_name_prefix + str(_fov_id+1) + f"_main_chr_2_{myo_df_prefix}.tif")
        
        if _analyze_laggards:    
            mask_savename.append(mask_name_prefix + str(_fov_id+1) + "_lagging_chr_dna.tif")
            mask_savename.append(mask_name_prefix + str(_fov_id+1) + "_lagging_chr_dna_rim.tif")
            mask_savename.append(mask_name_prefix + str(_fov_id+1) + "_lagging_chr_er.tif")
            if analyze_myo == True:
                mask_savename.append(mask_name_prefix + str(_fov_id+1) + f"_lagging_chr_{myo_df_prefix}.tif")
        
        if len(mask_savename) == len(_mask_each_fov):
            print ('Saving segmentation masks.')
            for _mask_names, _mask in zip(mask_savename, _mask_each_fov):
                if os.path.exists(mask_save_path + os.sep + _mask_names):
                    os.remove(mask_save_path + os.sep + _mask_names)
                io.imsave(mask_save_path + os.sep + _mask_names, skimage.img_as_ubyte(_mask), check_contrast=False)

        
        ############################################################
        # Add (other) shared cell and time information
        
        # measure dist between two chromosome mass
        label_chr_mass_mask = label(dna_mask)
        props = regionprops_table(label_chr_mass_mask,properties=['centroid'])
        # for two objects
        if len(props['centroid-0'])==2:
            _X1, _Y1 = props['centroid-0'][0],props['centroid-1'][0]
            _X2, _Y2 = props['centroid-0'][1],props['centroid-1'][1]
            _dist =  np.linalg.norm(np.array([_X1, _Y1])-np.array([_X2, _Y2]))
        # for one object
        elif len(props['centroid-0'])==1:
            _dist = 0
        else:
            _dist=np.nan
        
        _measurement_df_each_fov['chromosome_segreg_dist'] = _dist
        _measurement_df_each_fov.insert(0, 'cell_id', cell_id)
        _measurement_df_each_fov.insert(1, 'time_interval',time_interval)
        _measurement_df_each_fov.insert(1, 'timepoint', _fov_id + 1)
        _measurement_df_each_fov.insert(0, 'data_path', data_saved_folder.split("\\")[-1])
        
        # Add to the all fov results
        _measurement_df_all_fov = pd.concat([_measurement_df_all_fov,_measurement_df_each_fov])
         
        print (f'Analysis for TIMEPOINT_{_fov_id+1} is complete.')
        
    #============================= save results as excel for all timepoints ==========================================#
    # Export measurements to excel
    analysis_save_path =data_analysis_folder +os.sep+"measurements"
    if not os.path.exists(analysis_save_path):
        os.makedirs(analysis_save_path)
    
    
    if not _separate_inner_core_segment_measurement:
        analysis_savename = f'measurement_for_{cell_id}.xlsx'
        if not os.path.exists(analysis_save_path):
            os.makedirs(analysis_save_path)
        if os.path.exists(analysis_save_path+os.sep+analysis_savename) is False:
            _measurement_df_all_fov.to_excel(analysis_save_path+os.sep+analysis_savename,index=False)
        else:
            if not overwrite_measurement:
                analysis_savename = f'new_measurement_for_{cell_id}.xlsx'
                _measurement_df_all_fov.to_excel(analysis_save_path+os.sep+analysis_savename,index=False)
            else:
                _measurement_df_all_fov.to_excel(analysis_save_path+os.sep+analysis_savename,index=False)
    
    else:
        # integrated chr objects
        _measurement_df_all_fov_whole = (_measurement_df_all_fov[_measurement_df_all_fov['inner_core_segment_number'].
                                                                 str.contains('all')==True])
        
        analysis_savename_whole = f'measurement_for_{cell_id}_integrated.xlsx'
        if os.path.exists(analysis_save_path+os.sep+analysis_savename_whole) is False:
            _measurement_df_all_fov_whole.to_excel(analysis_save_path+os.sep+analysis_savename_whole,index=False)
        else:
            analysis_savename = f'new_measurement_for_{cell_id}_integrated.xlsx'
            _measurement_df_all_fov_whole.to_excel(analysis_save_path+os.sep+analysis_savename_whole,index=False)
            
        # binned chr objects
        _measurement_df_all_fov_bins = (_measurement_df_all_fov[_measurement_df_all_fov['inner_core_segment_number'].
                                                                 str.contains('all')!=True])
        analysis_savename_bins = f'measurement_for_{cell_id}_bins.xlsx'
        if os.path.exists(analysis_save_path+os.sep+analysis_savename_bins) is False:
            _measurement_df_all_fov_bins.to_excel(analysis_save_path+os.sep+analysis_savename_bins,index=False)
        else:
            if not overwrite_measurement:
                analysis_savename = f'new_measurement_for_{cell_id}_bins.xlsx'
                _measurement_df_all_fov_bins.to_excel(analysis_save_path+os.sep+analysis_savename_bins,index=False)
            else:
                _measurement_df_all_fov_bins.to_excel(analysis_save_path+os.sep+analysis_savename_bins,index=False)
        
        
        
        
    print(f"Measurement for cell {cell_id} saved.")
    print("Analysis completed in %.2f seconds." % (time.time() - start_time))  
    print ('-----------------------------------------------------------------------------------------------------')
    print ('-----------------------------------------------------------------------------------------------------')
    
    

Starting analysis for cell_cell1
-----------------------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------------------
There are 4 fovs/timepoints for this 4-channel, 4-slice dataset.
Number of masks provided do not match number of fovs/timepoints. Use saved masks for timepoints that have match only.
Midboxes dict with 1 bins are generated.
-- group all objects into one
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
-- Analyze TIMEPOINT 1
No saved mask. Use the default empty mask.
-- Analyze the main chromosome masses.
Segmentation of chromosome using otsu (or other defualt) method.
Midboxes dict with 5 bins are generated.


C:\Users\Shiwei\Documents\AnalysisTool\mitosis_analysis.py:327: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if mask_file [-3:] == 'roi':
C:\Users\Shiwei\Documents\AnalysisTool\mitosis_analysis.py:338: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if mask_file [-3:] == 'tif':


There are 441 small membrane fragments (sigma = 2) removed after converting detected ridges to binary images.
Actual non-ER channel is NUP. Use the proper segmentation accordingly.
Using 157.43 to segment the myo enriched region.
There are 441 small membrane fragments (sigma = 2) removed after converting detected ridges to binary images.
Actual non-ER channel is NUP. Use the proper segmentation accordingly.
Using 157.43 to segment the myo enriched region.
-- Analyze the lagging chromosomes.
Actual non-ER channel is NUP. Use the proper segmentation accordingly.
Using 157.43 to segment the myo enriched region.
There are 441 small membrane fragments (sigma = 2) removed after converting detected ridges to binary images.
Saving segmentation masks.
Analysis for TIMEPOINT_1 is complete.
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
-- Analyze TIMEPOINT 2
No saved mask. Use the default empty mask.
-- Analyze the main chromosome masses.
Segmentation of chrom

Using 138.46 to segment the myo enriched region.
There are 41 small membrane fragments (sigma = 2) removed after converting detected ridges to binary images.
Saving segmentation masks.
Analysis for TIMEPOINT_1 is complete.
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
-- Analyze TIMEPOINT 2
No saved mask. Use the default empty mask.
-- Analyze the main chromosome masses.
Segmentation of chromosome using otsu (or other defualt) method.
Midboxes dict with 5 bins are generated.
There are 67 small membrane fragments (sigma = 2) removed after converting detected ridges to binary images.
Actual non-ER channel is NUP. Use the proper segmentation accordingly.
Using 138.21 to segment the myo enriched region.
There are 67 small membrane fragments (sigma = 2) removed after converting detected ridges to binary images.
Actual non-ER channel is NUP. Use the proper segmentation accordingly.
Using 138.21 to segment the myo enriched region.
-- Analyze the lagging ch

## 5.2. Cell associated analysis

In [31]:
# Analyze cell global information
# ER-associated analysis of the cell (with DNA exclusion)

for _index, _image_list in enumerate(sel_image_list_combined[:]):
    
    # Reading cell info and determine methods based on cell info:
    _cell_info = sel_cell_info.iloc[_index]
    cell_id = _cell_info['cell_id']
    
    if np.isnan(_cell_info['cell_center_x']) or np.isnan(_cell_info['cell_center_x']):
        cell_center = [750,750]
        threshold_by_region_method = 'GLOBAL'
    else:    
        cell_center = [round(_cell_info['cell_center_x']),round(_cell_info['cell_center_y'])]
        threshold_by_region_method = 'USER'

    if not np.isnan(_cell_info['cell_size']):
        cell_size = round(_cell_info['cell_size'])
    else:
        cell_size = 600
        
    if not np.isnan(_cell_info['z_selected']):
        z_selected = round(_cell_info['z_selected'])
        _use_pre_defined_Z = True
    else:
        z_selected = 1
        _use_pre_defined_Z = False
    
    
    if type(_cell_info['dna_masks']) is str:
        _mask_file_folder = data_saved_folder + os.sep+ _cell_info['dna_masks']
        if os.path.exists(_mask_file_folder):
            _use_saved_mask = True
            #_use_saved_mask = False test purposes
        else:
            ("Cell info contains mask path, but no valid mask path/folder actually exists.")
            _mask_file_folder = ''
            _use_saved_mask = False
            
    else:
        _mask_file_folder = ''
        _use_saved_mask = False
        
#########################################################################################################################
#########################################################################################################################
    # Analyze each cell below.
    print (f'Starting analysis for cell_{cell_id}')
    print ('-----------------------------------------------------------------------------------------------------')
    print ('-----------------------------------------------------------------------------------------------------')
    import time
    start_time = time.time()
    
    #================================load images and massk for all timepoints========================================#

    # load metamorph images for each cell
    sorted_mm_data_array = load_metamorph_tifs_multi_z (_image_list, 
                                                       num_of_ch=num_of_ch, 
                                                       ch_to_be_analyzed = ch_to_be_analyzed,
                                                       timelapsed = timelapsed,
                                                       exclude_transmitted=exclude_transmitted)
    
        
    # generate mask list depending if the used saved mask files are correct.
    mask_filename_list = generate_mask_filename_list_for_timelapsed (len(sorted_mm_data_array),
                                                                     mask_file_folder=_mask_file_folder, 
                                                                          use_saved_mask = _use_saved_mask)
    
    # re-define channel index after channel exclusion
    new_dna_ch_index=ch_index_after_ch_exclusion (dna_ch_index,num_of_ch,ch_to_be_analyzed)
    new_myo_ch_index =ch_index_after_ch_exclusion (myo_ch_index,num_of_ch,ch_to_be_analyzed)
    new_er_ch_index =ch_index_after_ch_exclusion (er_ch_index,num_of_ch,ch_to_be_analyzed)
    new_cell_ch_index=ch_index_after_ch_exclusion (cell_ch_index,num_of_ch,ch_to_be_analyzed)
    

    # set df for all timepoints
    _measurement_df_all_fov = pd.DataFrame(columns=cell_measurement_columns)
    
    #================================analyze each timepoint========================================#
     
    # _fov_id ~ _timepoint_index
    for _fov_id in range(len(sorted_mm_data_array)):
        
        
        print ('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
        print (f'-- Analyze TIMEPOINT {_fov_id + 1}')
                            
        _measurement_df_each_fov = pd.DataFrame() # use empty column so new measurements can be directly appended below                    
                            
        # set mask list to save
        _mask_each_fov = []
        
        # load image data for the corresponding timepoint
        ims_each_fov_3D = sorted_mm_data_array [_fov_id]

        
        # get pseudo or saved dna mask
        raw_image_shape = ims_each_fov_3D[0,0,:,:].shape
        saved_dna_mask = generate_masks_from_saved (mask_filename_list[_fov_id],raw_image_shape)

        # crop saved_mask if necessary
        if threshold_by_region_method == "USER":
            saved_dna_mask = saved_dna_mask [int(cell_center[1]-cell_size/2):int(cell_center[1]+cell_size/2),  #crop x,y
                         int(cell_center[0]-cell_size/2):int(cell_center[0]+cell_size/2)] 
            
                
        # crop raw images if necessary
        if threshold_by_region_method == "USER":
            ims_each_fov_3D = ims_each_fov_3D[:,:,
                        int(cell_center[1]-cell_size/2):int(cell_center[1]+cell_size/2),    # channel crop x
                         int(cell_center[0]-cell_size/2):int(cell_center[0]+cell_size/2)]   # channel crop y
        
        
        ############################################################
        # Analyze the channel for the whole cell and cortex:
        print ('-- Analyze the cell.')
        cell_mask_each_fov = []
        
        if _use_pre_defined_Z: 
            # load xy for all channels for the pre-specified Z
            ims_ch_xy = ims_each_fov_3D [:,z_selected-1,:,:]
            
            _measurement_df_each_fov['object_z_slice']=[z_selected]
            
            
            # er mask for the cell
            if analyze_er:
                _all_er_mask = er_membrane_mask_segmentation (ims_ch_xy,
                                                     er_ch_index =new_er_ch_index, 
                                                     cell_center=cell_center, cell_size=cell_size,
                                             threshold_by_region = 'GLOBAL',ridge_sigma=ridge_sigma,
                                                         sm_fragment_size =sm_fragment_size,
                                                             non_ridge_segmentation=non_ridge_segmentation,
                                                             enriched_membrane=enriched_membrane) 
            else:
                print('No ER analysis, return empty values.')
                _all_er_mask = np.zeros(ims_ch_xy[0].shape)
            
            
            
            # all Nup mask
            if analyze_myo == True:
                if _actual_myo_ch == 'NUP' or _actual_myo_ch == 'MYO' or _actual_myo_ch == 'NE':
                    _all_nup_mask = myo_enrichment_mask_segmentation (ims_ch_xy,
                                                         myo_ch_index =new_myo_ch_index,
                                                         dna_mask = dna_mask_to_exclude,
                                                 cell_center=cell_center, cell_size=cell_size,
                                                         threshold_by_region = 'GLOBAL',
                                                num_of_obj = 2, std_ratio =std_ratio, 
                                                                 DNA_exclusion = DNA_exclusion_for_NUP)
                    
                elif _actual_myo_ch == 'ACTIN' or _actual_myo_ch == 'ER':
                    _all_nup_mask = er_membrane_mask_segmentation (ims_ch_xy,
                                                     er_ch_index =new_myo_ch_index, 
                                                     cell_center=cell_center, cell_size=cell_size,
                                                             sm_fragment_size = actin_fragment_size,
                                                                   ridge_sigma=actin_ridge_sigma,
                                             threshold_by_region = 'GLOBAL') 
                    
                
            else:
                print('No Nup (or Myosin) analysis, return empty values.')
                _all_nup_mask = np.zeros(ims_ch_xy[0].shape)


            
            # cell mask for the cell
            if analyze_myo:
                _cell_mask = cell_mask_segmentation(ims_ch_xy, new_cell_ch_index, 
                                                segmentation_by_percentile = True, cytoplasm_ratio= cytoplasm_ratio)
            else:
                _cell_mask = cell_mask_segmentation(ims_ch_xy, new_cell_ch_index, 
                                                segmentation_by_percentile = False, cytoplasm_ratio= cytoplasm_ratio)
                _cell_mask = dilation (_cell_mask, disk(3))
                _cell_mask = skimage.filters.median (_cell_mask, disk(15))
                _cell_mask = erosion (_cell_mask, disk(3))
                
                
            _cell_mask_inner = erosion (_cell_mask, disk(10)) 
            
            
            # nondna area (excluding laggards as wellif there are ) to exclude dna-associated components\
            if not inverse_dna:
                all_dna_mask = chromosome_mask_segmentation(ims_ch_xy,
                                                    dna_ch_index=new_dna_ch_index, 
                                                    cell_mask_ch_index =new_myo_ch_index, cell_center=cell_center,
                                                    cell_size=cell_size, check_chrom_num = False, 
                                                        small_object_th=50,
                                         threshold_by_region = "GLOBAL",inner_core_mode = False)
                
            else:
                all_dna_mask = inverse_chromosome_mask_segmentation_by_line_profile(ims_ch_xy,
                                                    dna_ch_index=new_dna_ch_index, 
                                                             cell_center=cell_center,
                                                             cell_size=cell_size, 
                                                            extra_erosion_factor = extra_erosion_factor,
                                         threshold_by_region = "GLOBAL",inner_core_mode = False, 
                                                non_chr_low_ratio =0.09, chr_peak_prominence = 10,
                                                    chr_mask_from_saved = saved_dna_mask)
                
                
            nondna_area = erosion(all_dna_mask ==0, disk(nup_core_thickness+1))
            
            ####################################
            # get mask for each subcelluar-area
            cell_er_mask = _cell_mask_inner * _all_er_mask
            
            # 1. all non-dna adjacent ER and ER-associated-Nup
            _nondna_er_mask = cell_er_mask * nondna_area
            _nondna_er_nup = _all_nup_mask * dilation(_nondna_er_mask, disk(_er_nup_overlap_factor)) * nondna_area
            
            # measurement for all non-dna endoplasmic area
            _measurement_df_each_fov[f'nondna_endo_{myo_df_prefix}_area']=np.sum(_nondna_er_nup)
            _measurement_df_each_fov['nondna_endo_er_area']=np.sum(_nondna_er_mask)
            _measurement_df_each_fov[f'nondna_endo_{myo_df_prefix}_normalized_area']=np.sum(_nondna_er_nup)/np.sum(_nondna_er_mask)
            
            # 2. all dna adjacent ER and Nup
            _dna_near_er_mask = cell_er_mask * (_nondna_er_mask==0)
            _dna_near_nup_mask = (_all_nup_mask * dilation(_nondna_er_mask, disk(_er_nup_overlap_factor))
                                  * (_nondna_er_nup==0))
            
            # measurement for all dna adjacent endoplasmic area
            _measurement_df_each_fov['dna_endo_er_area']=np.sum(_dna_near_er_mask)
            _measurement_df_each_fov[f'dna_endo_{myo_df_prefix}_area']=np.sum(_dna_near_nup_mask)
            _measurement_df_each_fov[f'dna_endo_{myo_df_prefix}_normalized_area']=np.sum(_dna_near_nup_mask)/np.sum(_dna_near_er_mask)
                                            
              
            # append masks above
            cell_mask_each_fov.append(_cell_mask)
            cell_mask_each_fov.append(_nondna_er_mask)
            cell_mask_each_fov.append(_nondna_er_nup)
            
            cell_mask_each_fov.append(_dna_near_er_mask)
            cell_mask_each_fov.append(_dna_near_nup_mask)

            ###############################################
            # Analyze the channel for the midzone:
            # generate main chromosome mask first 
            if not inverse_dna:
                dna_mask = chromosome_mask_segmentation(ims_ch_xy,
                                                    dna_ch_index=new_dna_ch_index, 
                                                             cell_mask_ch_index =new_myo_ch_index, cell_center=cell_center,
                                                             cell_size=cell_size, 
                                                    small_object_th =large_object_size,
                                         threshold_by_region = "GLOBAL",inner_core_mode = False, 
                                                    chr_mask_from_saved = saved_dna_mask)
            else:
                dna_mask = inverse_chromosome_mask_segmentation_by_line_profile(ims_ch_xy,
                                                    dna_ch_index=new_dna_ch_index, 
                                                             cell_center=cell_center,
                                                             cell_size=cell_size, 
                                                            extra_erosion_factor = extra_erosion_factor,
                                         threshold_by_region = "GLOBAL",inner_core_mode = False, 
                                                non_chr_low_ratio =0.09, chr_peak_prominence = 10,
                                                    chr_mask_from_saved = saved_dna_mask)
            
            # repeat the size filtering once with the same setting in case of type error occured in prior function
            dna_mask= dna_mask.astype(bool)
            dna_mask = remove_small_objects(dna_mask, large_object_size)
            
            labeled_main_chromosome, num_of_main_chromosome = ndi.label(dna_mask)
            
            if num_of_main_chromosome == 2:
                
                # measure dist between two chromosome mass
                label_chr_mass_mask = label(dna_mask)
                props = regionprops_table(label_chr_mass_mask,properties=['centroid'])
                _X1, _Y1 = props['centroid-0'][0],props['centroid-1'][0]
                _X2, _Y2 = props['centroid-0'][1],props['centroid-1'][1]
                _dist =  np.linalg.norm(np.array([_X1, _Y1])-np.array([_X2, _Y2]))
                
                # midzone
                refined_midbox_dict = generate_midbox_with_bins (dna_mask, num_of_bins=num_of_segments_for_cell,_verbose = True)
                # generate thinned DNA masks from z-max to help other analysis
                #dna_mask_to_exclude = erosion(dna_mask, disk(10))
                
                # midzone area using the central chromosome segment
                _midbox_midzone =refined_midbox_dict[np.ceil(
                    len(refined_midbox_dict)/2)-1] # use the middle segment for odd number of segments
                
                _center_midzone = _midbox_midzone * _cell_mask_inner
                
                
                # 3. midzone and periphery ER and Nup
                
                # midzone er and periphery er     
                _nondna_midzone_er = _nondna_er_mask * _center_midzone
                _nondna_peri_er = _nondna_er_mask * (_nondna_midzone_er == 0) 
                
                # get different subregions of endo-associated nup           
                # for midzone
                _nondna_midzone_endo_nup = _nondna_er_nup * _center_midzone 
                # for the rest of periphery
                _nondna_peri_endo_nup = _nondna_er_nup * (_nondna_midzone_endo_nup==0)
                
                # measurement for midzone area
                #_measurement_df_each_fov['midzone_er_area']=np.sum(_midzone_er)
                _measurement_df_each_fov['midzone_area']=np.sum(_center_midzone)
                _measurement_df_each_fov['midzone_nondna_er_normalized_area']=np.sum(_nondna_midzone_er)/np.sum(_center_midzone)
                
                _measurement_df_each_fov[f'midzone_nondna_endo_{myo_df_prefix}']=np.sum(_nondna_midzone_endo_nup)
                _measurement_df_each_fov['midzone_nondna_er_area']=np.sum(_nondna_midzone_er)
                _measurement_df_each_fov[f'midzone_endo_{myo_df_prefix}_normalized_area']=(np.sum(_nondna_midzone_endo_nup)/
                                                                              np.sum(_nondna_midzone_er))
                
                _measurement_df_each_fov[f'periphery_nondna_endo_{myo_df_prefix}']=np.sum(_nondna_peri_endo_nup)
                _measurement_df_each_fov['periphery_nondna_er_area']=np.sum(_nondna_peri_er)
                
                _measurement_df_each_fov[f'periphery_endo_{myo_df_prefix}_normalized_area']=(np.sum(_nondna_peri_endo_nup)/
                                                                              np.sum(_nondna_peri_er))
                
                # append midzone and related masks
                cell_mask_each_fov.append(_center_midzone)
                cell_mask_each_fov.append(_nondna_midzone_er)
                cell_mask_each_fov.append(_nondna_peri_er)
                cell_mask_each_fov.append(_nondna_midzone_endo_nup)
                cell_mask_each_fov.append(_nondna_peri_endo_nup)
                
                
            # if no valid chromosome segmentations
            else:     
                if num_of_main_chromosome == 1:
                    _dist = 0
                else:
                    _dist = np.nan
                _measurement_df_each_fov['midzone_er_area']=np.nan
                _measurement_df_each_fov['midzone_area']=np.nan
                _measurement_df_each_fov['midzone_nondna_er_normalized_area']=np.nan
                
                _measurement_df_each_fov[f'midzone_nondna_endo_{myo_df_prefix}']=np.nan
                _measurement_df_each_fov['midzone_nondna_er_area']=np.nan
                _measurement_df_each_fov[f'midzone_endo_{myo_df_prefix}_normalized_area']=np.nan

                _measurement_df_each_fov[f'periphery_nondna_endo_{myo_df_prefix}']=np.nan
                _measurement_df_each_fov['periphery_nondna_er_area']=np.nan
                
                _measurement_df_each_fov[f'periphery_endo_{myo_df_prefix}_normalized_area']=np.nan

        
            #######################################
            # Save cell mask images
            cell_mask_save_path =data_analysis_folder +os.sep+"cell_segmentations"+os.sep+cell_id
            
            if not os.path.exists(cell_mask_save_path):
                os.makedirs(cell_mask_save_path)
            
            cell_mask_savename = [] 
            if timelapsed == True:
                cell_mask_name_prefix = 'time_'
            else:
                cell_mask_name_prefix = 'fov_'
                    
                    
            cell_mask_savename.append(mask_name_prefix + str(_fov_id+1) + "_cell_mask.tif")
            cell_mask_savename.append(mask_name_prefix + str(_fov_id+1) + "_cell_nondna_endo_area.tif")
            cell_mask_savename.append(mask_name_prefix + str(_fov_id+1) + f"_cell_nondna_endo_{myo_df_prefix}.tif")
            
            cell_mask_savename.append(mask_name_prefix + str(_fov_id+1) + "_cell_dna_endo_area.tif")
            cell_mask_savename.append(mask_name_prefix + str(_fov_id+1) + f"_cell_dna_endo_{myo_df_prefix}.tif")            
            
            if num_of_main_chromosome == 2: # if there are valid midzone masks
                cell_mask_savename.append(mask_name_prefix + str(_fov_id+1) + "_cell_midzone_area.tif")
                cell_mask_savename.append(mask_name_prefix + str(_fov_id+1) + "_cell_midzone_nondna_er.tif")
                cell_mask_savename.append(mask_name_prefix + str(_fov_id+1) + "_cell_periphery_nondna_er.tif")
                cell_mask_savename.append(mask_name_prefix + str(_fov_id+1) + f"_cell_midzone_nondna_endo_{myo_df_prefix}.tif")
                cell_mask_savename.append(mask_name_prefix + str(_fov_id+1) + f"_cell_periphery_nondna_endo_{myo_df_prefix}.tif")

        
            if len(cell_mask_savename) == len(cell_mask_each_fov):
                print ('Saving segmentation masks.')
                for _mask_names, _mask in zip(cell_mask_savename, cell_mask_each_fov):
                    if os.path.exists(cell_mask_save_path + os.sep + _mask_names):
                        os.remove(cell_mask_save_path + os.sep + _mask_names)         
                    io.imsave(cell_mask_save_path + os.sep + _mask_names, skimage.img_as_ubyte(_mask), check_contrast=False)
            
            

        ############################################################
        # Add shared cell and time information
        _measurement_df_each_fov['chromosome_segreg_dist'] = _dist
        _measurement_df_each_fov.insert(0, 'cell_id', cell_id)
        _measurement_df_each_fov.insert(0, 'object_type', 'whole_cell')
        _measurement_df_each_fov.insert(1, 'time_interval',time_interval)
        _measurement_df_each_fov.insert(1, 'timepoint', _fov_id + 1)
        _measurement_df_each_fov.insert(0, 'data_path', data_saved_folder.split("\\")[-1])
        
        # Add to the all fov results
        _measurement_df_all_fov = pd.concat([_measurement_df_all_fov,_measurement_df_each_fov])
         
        print (f'Analysis for TIMEPOINT_{_fov_id+1} is complete.')            
            
            
    #============================= save results as excel for all timepoints ==========================================#
    # Export measurements to excel
    analysis_save_path =data_analysis_folder +os.sep+"measurements"
    if not os.path.exists(analysis_save_path):
        os.makedirs(analysis_save_path)
    
    
    if save_global_measurement:
        analysis_savename = f'measurement_for_{cell_id}_global_dna_excluded.xlsx'
        if not os.path.exists(analysis_save_path):
            os.makedirs(analysis_save_path)
        if os.path.exists(analysis_save_path+os.sep+analysis_savename) is False:
            _measurement_df_all_fov.to_excel(analysis_save_path+os.sep+analysis_savename,index=False)
        else:
            if not overwrite_measurement:
                analysis_savename = f'new_measurement_for_{cell_id}_bins.xlsx'
                _measurement_df_all_fov.to_excel(analysis_save_path+os.sep+analysis_savename,index=False)
            else:
                _measurement_df_all_fov.to_excel(analysis_save_path+os.sep+analysis_savename,index=False)
    
        
        
    print(f"Measurement for cell {cell_id} saved.")
    print("Analysis completed in %.2f seconds." % (time.time() - start_time))  
    print ('-----------------------------------------------------------------------------------------------------')
    print ('-----------------------------------------------------------------------------------------------------')

Starting analysis for cell_cell1
-----------------------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------------------
There are 4 fovs/timepoints for this 4-channel, 4-slice dataset.
Number of masks provided do not match number of fovs/timepoints. Use saved masks for timepoints that have match only.
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
-- Analyze TIMEPOINT 1
No saved mask. Use the default empty mask.
-- Analyze the cell.
There are 441 small membrane fragments (sigma = 2) removed after converting detected ridges to binary images.
Using 157.43 to segment the myo enriched region.
Segmentation of chromosome using otsu (or other defualt) method.
Midboxes dict with 3 bins are generated.
Saving segmentation masks.
Analysis for TIMEPOINT_1 is complete.
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

Using 155.79 to segment the myo enriched region.
Segmentation of chromosome using otsu (or other defualt) method.
Midboxes dict with 3 bins are generated.
Saving segmentation masks.
Analysis for TIMEPOINT_1 is complete.
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
-- Analyze TIMEPOINT 2
No saved mask. Use the default empty mask.
-- Analyze the cell.
There are 416 small membrane fragments (sigma = 2) removed after converting detected ridges to binary images.
Using 153.16 to segment the myo enriched region.
Segmentation of chromosome using otsu (or other defualt) method.
Midboxes dict with 3 bins are generated.
Saving segmentation masks.
Analysis for TIMEPOINT_2 is complete.
Measurement for cell cell4 saved.
Analysis completed in 4.87 seconds.
-----------------------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------------------


## 5.3. Cell associated Actin analysis (optional)

In [32]:
# Analyze cell global information part 2
# Actin for individual part of the cell
if _actual_myo_ch == 'ACTIN': 

    for _index, _image_list in enumerate(sel_image_list_combined[:]):
    
        # Reading cell info and determine methods based on cell info:
        _cell_info = sel_cell_info.iloc[_index]
        cell_id = _cell_info['cell_id']
    
        if np.isnan(_cell_info['cell_center_x']) or np.isnan(_cell_info['cell_center_x']):
            cell_center = [750,750]
            threshold_by_region_method = 'GLOBAL'
        else:    
            cell_center = [round(_cell_info['cell_center_x']),round(_cell_info['cell_center_y'])]
            threshold_by_region_method = 'USER'

        if not np.isnan(_cell_info['cell_size']):
            cell_size = round(_cell_info['cell_size'])
        else:
            cell_size = 600
        
        if not np.isnan(_cell_info['z_selected']):
            z_selected = round(_cell_info['z_selected'])
            _use_pre_defined_Z = True
        else:
            z_selected = 1
            _use_pre_defined_Z = False
    
    
        if type(_cell_info['dna_masks']) is str:
            _mask_file_folder = data_saved_folder + os.sep+ _cell_info['dna_masks']
            if os.path.exists(_mask_file_folder):
                _use_saved_mask = True
                #_use_saved_mask = False test purposes
            else:
                ("Cell info contains mask path, but no valid mask path/folder actually exists.")
                _mask_file_folder = ''
                _use_saved_mask = False
            
        else:
            _mask_file_folder = ''
            _use_saved_mask = False
        
#########################################################################################################################
#########################################################################################################################
       # Analyze each cell below.
        print (f'Starting analysis for cell_{cell_id}')
        print ('-----------------------------------------------------------------------------------------------------')
        print ('-----------------------------------------------------------------------------------------------------')
        import time
        start_time = time.time()
    
        #================================load images and massk for all timepoints========================================#

        # load metamorph images for each cell
        sorted_mm_data_array = load_metamorph_tifs_multi_z (_image_list, 
                                                       num_of_ch=num_of_ch, 
                                                       ch_to_be_analyzed = ch_to_be_analyzed,
                                                       timelapsed = timelapsed,
                                                       exclude_transmitted=exclude_transmitted)
    
        
        # generate mask list depending if the used saved mask files are correct.
        mask_filename_list = generate_mask_filename_list_for_timelapsed (len(sorted_mm_data_array),
                                                                     mask_file_folder=_mask_file_folder, 
                                                                          use_saved_mask = _use_saved_mask)
    
        # re-define channel index after channel exclusion
        new_dna_ch_index=ch_index_after_ch_exclusion (dna_ch_index,num_of_ch,ch_to_be_analyzed)
        new_myo_ch_index =ch_index_after_ch_exclusion (myo_ch_index,num_of_ch,ch_to_be_analyzed)
        new_er_ch_index =ch_index_after_ch_exclusion (er_ch_index,num_of_ch,ch_to_be_analyzed)
        new_cell_ch_index=ch_index_after_ch_exclusion (cell_ch_index,num_of_ch,ch_to_be_analyzed)
    

        # set df for all timepoints
        _measurement_df_all_fov = pd.DataFrame(columns=cell_actin_measurement_columns)
    
    #================================analyze each timepoint========================================#
     
        # _fov_id ~ _timepoint_index
        for _fov_id in range(len(sorted_mm_data_array)):
        
        
            print ('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
            print (f'-- Analyze TIMEPOINT {_fov_id + 1}')
                            
            _measurement_df_each_fov = pd.DataFrame() # use empty column so new measurements can be directly appended below                    
                            
            # set mask list to save
            _mask_each_fov = []
        
            # load image data for the corresponding timepoint
            ims_each_fov_3D = sorted_mm_data_array [_fov_id]

        
            # get pseudo or saved dna mask
            raw_image_shape = ims_each_fov_3D[0,0,:,:].shape
            saved_dna_mask = generate_masks_from_saved (mask_filename_list[_fov_id],raw_image_shape)

            # crop saved_mask if necessary
            if threshold_by_region_method == "USER":
                saved_dna_mask = saved_dna_mask [int(cell_center[1]-cell_size/2):int(cell_center[1]+cell_size/2),  #crop x,y
                         int(cell_center[0]-cell_size/2):int(cell_center[0]+cell_size/2)] 
            
                
            # crop raw images if necessary
            if threshold_by_region_method == "USER":
                ims_each_fov_3D = ims_each_fov_3D[:,:,
                        int(cell_center[1]-cell_size/2):int(cell_center[1]+cell_size/2),    # channel crop x
                         int(cell_center[0]-cell_size/2):int(cell_center[0]+cell_size/2)]   # channel crop y
        
        
            ############################################################
            # Analyze the channel for the whole cell and cortex:
            print ('-- Analyze the cell.')
            cell_mask_each_fov = []
        
            if _use_pre_defined_Z: 
                # load xy for all channels for the pre-specified Z
                ims_ch_xy = ims_each_fov_3D [:,z_selected-1,:,:]
            
                _measurement_df_each_fov['object_z_slice']=[z_selected]
            
            
                # er mask for the cell
                if analyze_er:
                    _all_er_mask = er_membrane_mask_segmentation (ims_ch_xy,
                                                     er_ch_index =new_er_ch_index, 
                                                     cell_center=cell_center, cell_size=cell_size,
                                             threshold_by_region = 'GLOBAL',ridge_sigma=ridge_sigma,
                                                         sm_fragment_size =sm_fragment_size,
                                                             non_ridge_segmentation=non_ridge_segmentation,
                                                             enriched_membrane=enriched_membrane) 
                else:
                    print('No ER analysis, return empty values.')
                    _all_er_mask = np.zeros(ims_ch_xy[0].shape)
            
            
            
                # all Nup mask
                if analyze_myo == True:
                    if _actual_myo_ch == 'NUP' or _actual_myo_ch == 'MYO' or _actual_myo_ch == 'NE':
                        _all_nup_mask = myo_enrichment_mask_segmentation (ims_ch_xy,
                                                         myo_ch_index =new_myo_ch_index,
                                                         dna_mask = dna_mask_to_exclude,
                                                 cell_center=cell_center, cell_size=cell_size,
                                                         threshold_by_region = 'GLOBAL',
                                                num_of_obj = 2, std_ratio =std_ratio, 
                                                                 DNA_exclusion = DNA_exclusion_for_NUP)
                    
                    elif _actual_myo_ch == 'ACTIN' or _actual_myo_ch == 'ER':
                        _all_nup_mask = er_membrane_mask_segmentation (ims_ch_xy,
                                                     er_ch_index =new_myo_ch_index, 
                                                     cell_center=cell_center, cell_size=cell_size,
                                                             sm_fragment_size = actin_fragment_size,
                                                                   ridge_sigma=actin_ridge_sigma,
                                             threshold_by_region = 'GLOBAL') 
                    
                
                else:
                    print('No Nup (or Myosin) analysis, return empty values.')
                    _all_nup_mask = np.zeros(ims_ch_xy[0].shape)


            
                # cell mask for the cell
                if analyze_myo:
                    _cell_mask = cell_mask_segmentation(ims_ch_xy, new_cell_ch_index, 
                                                segmentation_by_percentile = True, cytoplasm_ratio= cytoplasm_ratio)
                else:
                    _cell_mask = cell_mask_segmentation(ims_ch_xy, new_cell_ch_index, 
                                                segmentation_by_percentile = False, cytoplasm_ratio= cytoplasm_ratio)
                    _cell_mask = dilation (_cell_mask, disk(3))
                    _cell_mask = skimage.filters.median (_cell_mask, disk(15))
                    _cell_mask = erosion (_cell_mask, disk(3))
                
                
                _cell_mask_inner = erosion (_cell_mask, disk(10)) 
                
                # get mask for each sub-area
                # re-define nup mask as actin mask for script below
                _all_actin_mask =_all_nup_mask
                # cortex
                _cortical_mask = _cell_mask * (_cell_mask_inner ==0)
                _cortical_actin = _all_actin_mask * _cortical_mask
                # endo
                _endo_er_mask = _cell_mask_inner * _all_er_mask
                _endo_actin = _all_actin_mask *_endo_er_mask 
                
                # measurement for cortex area
                _measurement_df_each_fov[f'cortical_{myo_df_prefix}_area']=np.sum(_cortical_actin)
                _measurement_df_each_fov['cortical_rim_area']=np.sum(_cortical_mask)
                _measurement_df_each_fov[f'cortical_{myo_df_prefix}_normalized_area']=np.sum(_cortical_actin)/np.sum(_cortical_mask)
                
                # measurement for endoplasmic area
                _measurement_df_each_fov[f'endo_{myo_df_prefix}_area']=np.sum(_endo_actin)
                _measurement_df_each_fov['endo_area']=np.sum(_endo_er_mask)
                _measurement_df_each_fov[f'endo_{myo_df_prefix}_normalized_area']=np.sum(_endo_actin)/np.sum(_endo_er_mask)

            
                cell_mask_each_fov.append(_cell_mask)
                cell_mask_each_fov.append(_all_er_mask)
                cell_mask_each_fov.append(_all_actin_mask)
                cell_mask_each_fov.append(_endo_actin)
                cell_mask_each_fov.append(_cortical_actin)
                

            ###############################################
                # Analyze the channel for the midzone:
                # generate main chromosome mask 
                if inverse_dna:
                    dna_mask = inverse_chromosome_mask_segmentation_by_line_profile(ims_ch_xy,
                                                    dna_ch_index=new_dna_ch_index, 
                                                             cell_center=cell_center,
                                                             cell_size=cell_size, 
                                         threshold_by_region = "GLOBAL",inner_core_mode = False, 
                                                non_chr_low_ratio =0.09, chr_peak_prominence = 10,
                                                    chr_mask_from_saved = saved_dna_mask)
                else:
                    dna_mask = chromosome_mask_segmentation(ims_ch_xy,
                                                    dna_ch_index=new_dna_ch_index, 
                                                             cell_mask_ch_index =new_myo_ch_index, cell_center=cell_center,
                                                             cell_size=cell_size, 
                                                    small_object_th =large_object_size,
                                         threshold_by_region = "GLOBAL",inner_core_mode = False, 
                                                    chr_mask_from_saved = saved_dna_mask)
                    
                # repeat the size filtering once with the same setting in case of unknown error occured in prior function
                dna_mask= dna_mask.astype(bool)
                dna_mask = remove_small_objects(dna_mask, large_object_size)
            
                labeled_main_chromosome, num_of_main_chromosome = ndi.label(dna_mask)
            
                if num_of_main_chromosome == 2:
                
                    # measure dist between two chromosome mass
                    label_chr_mass_mask = label(dna_mask)
                    props = regionprops_table(label_chr_mass_mask,properties=['centroid'])
                    _X1, _Y1 = props['centroid-0'][0],props['centroid-1'][0]
                    _X2, _Y2 = props['centroid-0'][1],props['centroid-1'][1]
                    _dist =  np.linalg.norm(np.array([_X1, _Y1])-np.array([_X2, _Y2]))
                
                    # midzone
                    refined_midbox_dict = generate_midbox_with_bins (dna_mask, num_of_bins=num_of_segments_for_cell,_verbose = True)
                    # generate thinned DNA masks from z-max to help other analysis
                    #dna_mask_to_exclude = erosion(dna_mask, disk(10))
                
                    _midbox_midzone =refined_midbox_dict[np.ceil(
                    len(refined_midbox_dict)/2)-1] # use the middle segment for odd number of segments
                
                    _center_midzone = _midbox_midzone * _cell_mask_inner
                    # midzone actin and er
                    _midzone_actin = _all_actin_mask * _center_midzone
                    _midzone_er = _all_er_mask * _center_midzone
                
                    # measurement for midzone area
                    _measurement_df_each_fov[f'midzone_{myo_df_prefix}_area']=np.sum(_midzone_actin)
                    _measurement_df_each_fov['midzone_er_area']=np.sum(_midzone_er)
                    _measurement_df_each_fov['midzone_area']=np.sum(_center_midzone)
                    _measurement_df_each_fov[f'midzone_{myo_df_prefix}_normalized_area']=np.sum(_midzone_actin)/np.sum(_center_midzone)
                    _measurement_df_each_fov['midzone_er_normalized_area']=np.sum(_midzone_er)/np.sum(_center_midzone)
                
                    # append midzone and related masks
                    cell_mask_each_fov.append(_center_midzone)
                    cell_mask_each_fov.append(_midzone_er)
                    cell_mask_each_fov.append(_midzone_actin)
                
                # if no valid chromosome segmentations
                else:             
                    _dist = np.nan
                    _measurement_df_each_fov[f'midzone_{myo_df_prefix}_area']=np.nan
                    _measurement_df_each_fov['midzone_er_area']=np.nan
                    _measurement_df_each_fov['midzone_area']=np.nan
                    _measurement_df_each_fov[f'midzone_{myo_df_prefix}_normalized_area']=np.nan
                    _measurement_df_each_fov['midzone_er_normalized_area']=np.nan
        
            #######################################
                # Save cell mask images
                cell_mask_save_path =data_analysis_folder +os.sep+f"cell_{myo_df_prefix}_segmentations"+os.sep+cell_id
            
                if not os.path.exists(cell_mask_save_path):
                    os.makedirs(cell_mask_save_path)
            
                cell_mask_savename = [] 
                if timelapsed == True:
                    cell_mask_name_prefix = 'time_'
                else:
                    cell_mask_name_prefix = 'fov_'
                    
                    
                cell_mask_savename.append(mask_name_prefix + str(_fov_id+1) + "_cell_mask.tif")
                cell_mask_savename.append(mask_name_prefix + str(_fov_id+1) + "_cell_endo_area.tif")
                cell_mask_savename.append(mask_name_prefix + str(_fov_id+1) + f"_cell_all_{myo_df_prefix}.tif")
                cell_mask_savename.append(mask_name_prefix + str(_fov_id+1) + f"_cell_endo_{myo_df_prefix}.tif")
                cell_mask_savename.append(mask_name_prefix + str(_fov_id+1) + f"_cell_cortical_{myo_df_prefix}.tif")
            
                if num_of_main_chromosome == 2: # if there are valid midzone masks
                    cell_mask_savename.append(mask_name_prefix + str(_fov_id+1) + "_cell_midzone_area.tif")
                    cell_mask_savename.append(mask_name_prefix + str(_fov_id+1) + "_cell_midzone_er.tif")
                    cell_mask_savename.append(mask_name_prefix + str(_fov_id+1) + f"_cell_midzone_{myo_df_prefix}.tif")

        
                if len(cell_mask_savename) == len(cell_mask_each_fov):
                    print ('Saving segmentation masks.')
                    for _mask_names, _mask in zip(cell_mask_savename, cell_mask_each_fov):
                        if os.path.exists(cell_mask_save_path + os.sep + _mask_names):
                            os.remove(cell_mask_save_path + os.sep + _mask_names) 
                        io.imsave(cell_mask_save_path + os.sep + _mask_names, skimage.img_as_ubyte(_mask), check_contrast=False)
            
            

        ############################################################
            # Add shared cell and time information
            _measurement_df_each_fov['chromosome_segreg_dist'] = _dist
            _measurement_df_each_fov.insert(0, 'cell_id', cell_id)
            _measurement_df_each_fov.insert(0, 'object_type', 'whole_cell')
            _measurement_df_each_fov.insert(1, 'time_interval',time_interval)
            _measurement_df_each_fov.insert(1, 'timepoint', _fov_id + 1)
            _measurement_df_each_fov.insert(0, 'data_path', data_saved_folder.split("\\")[-1])
        
            # Add to the all fov results
            _measurement_df_all_fov = pd.concat([_measurement_df_all_fov,_measurement_df_each_fov])
         
            print (f'Analysis for TIMEPOINT_{_fov_id+1} is complete.')            
            
            
    #============================= save results as excel for all timepoints ==========================================#
        # Export measurements to excel
        analysis_save_path =data_analysis_folder +os.sep+"measurements"
        if not os.path.exists(analysis_save_path):
            os.makedirs(analysis_save_path)
    
    
        if save_global_measurement:
            analysis_savename = f'measurement_for_{cell_id}_global_{myo_df_prefix}.xlsx'
            if not os.path.exists(analysis_save_path):
                os.makedirs(analysis_save_path)
            if os.path.exists(analysis_save_path+os.sep+analysis_savename) is False:
                _measurement_df_all_fov.to_excel(analysis_save_path+os.sep+analysis_savename,index=False)
            else:
                analysis_savename = f'new_measurement_for_{cell_id}_{myo_df_prefix}.xlsx'
                _measurement_df_all_fov.to_excel(analysis_save_path+os.sep+analysis_savename,index=False)
    
        
        
        print(f"Measurement for cell {cell_id} saved.")
        print("Analysis completed in %.2f seconds." % (time.time() - start_time))  
        print ('-----------------------------------------------------------------------------------------------------')
        print ('-----------------------------------------------------------------------------------------------------')
              
            
            
else:
    print('Skip this analysis because it is not relevant for the experiment.')                

Skip this analysis because it is not relevant for the experiment.


## 5.4 Core and noncore analysis

In [33]:
# Perform image segmentation and analysis for all data within the data_save_folder.
# Check, Edit, and Re-run all the boxes above if errors occur.
# Segmentation masks will overwrite everytime when it runs; 
# measurements will be saved as a new excel for the first run; for later runs, measurements will overwrite old ones.

# define core_noncore_measurement_columns for old notebooks where this has not been set above
try:
    core_noncore_measurement_columns
except NameError:
    # Set core_noncore_measurement_columns if this has not be set above
    core_noncore_measurement_columns =['data_path','cell_id','timepoint',
                            'object_type', 'object_id','object_z_slice',   
                                   # er channel measurements
                                   'er_recruit_normalized_area', 
                                   'er_recruit_area',
                                  'er_dna_rim',
                                   # myo/nup channel measurements
                                    f'{myo_df_prefix}_recruit_normalized_area',
                                   f'{myo_df_prefix}_recruit_area',
                                   f'{myo_df_prefix}_dna_rim',
                                   # core or noncore info
                                   'core_noncore_type']


for _index, _image_list in enumerate(sel_image_list_combined[:]):
    
    # Reading cell info and determine methods based on cell info:
    _cell_info = sel_cell_info.iloc[_index]
    
    cell_id = _cell_info['cell_id']
    
    if np.isnan(_cell_info['cell_center_x']) or np.isnan(_cell_info['cell_center_x']):
        cell_center = [750,750]
        threshold_by_region_method = 'GLOBAL'
    else:    
        cell_center = [round(_cell_info['cell_center_x']),round(_cell_info['cell_center_y'])]
        threshold_by_region_method = 'USER'

    if not np.isnan(_cell_info['cell_size']):
        cell_size = round(_cell_info['cell_size'])
    else:
        cell_size = 600
        
    if not np.isnan(_cell_info['z_selected']):
        z_selected = round(_cell_info['z_selected'])
        _use_pre_defined_Z = True
    else:
        z_selected = 1
        _use_pre_defined_Z = False
    
    
    if type(_cell_info['dna_masks']) is str:
        _mask_file_folder = data_saved_folder + os.sep+ _cell_info['dna_masks']
        if os.path.exists(_mask_file_folder):
            _use_saved_mask = True
        else:
            ("Cell info contains mask path, but no valid mask path/folder actually exists.")
            _mask_file_folder = ''
            _use_saved_mask = False
            
    else:
        _mask_file_folder = ''
        _use_saved_mask = False
        
#########################################################################################################################
#########################################################################################################################
    # Analyze each cell below.
    print (f'Starting analysis for cell_{cell_id}')
    print ('-----------------------------------------------------------------------------------------------------')
    print ('-----------------------------------------------------------------------------------------------------')
    import time
    start_time = time.time()
    
    #================================load images and massk for all timepoints========================================#
    
    
    # load metamorph images for each cell
    sorted_mm_data_array = load_metamorph_tifs_multi_z (_image_list, 
                                                       num_of_ch=num_of_ch, 
                                                       ch_to_be_analyzed = ch_to_be_analyzed,
                                                       timelapsed = timelapsed,
                                                       exclude_transmitted=exclude_transmitted)
    
        
    # generate mask list depending if the used saved mask files are correct.
    mask_filename_list = generate_mask_filename_list_for_timelapsed (len(sorted_mm_data_array),
                                                                     mask_file_folder=_mask_file_folder, 
                                                                          use_saved_mask = _use_saved_mask)
    
    # re-define channel index after channel exclusion
    new_dna_ch_index=ch_index_after_ch_exclusion (dna_ch_index,num_of_ch,ch_to_be_analyzed)
    new_myo_ch_index =ch_index_after_ch_exclusion (myo_ch_index,num_of_ch,ch_to_be_analyzed)
    new_er_ch_index =ch_index_after_ch_exclusion (er_ch_index,num_of_ch,ch_to_be_analyzed)
    
    # get object using 3D mask if DNA is stained positively
    if not inverse_dna: 
        # generate 3D mask for main chromosome mass
        _main_chromosome_mask = object_segmentation_timelapsed_3D (sorted_mm_data_array, object_ch_index = new_dna_ch_index, 
                            threshold_method = skimage.filters.thresholding.threshold_otsu, 
                            threshold_by_region = threshold_by_region_method, cell_center = cell_center, cell_size = cell_size,
                            large_object_size = large_object_size, object_type = "main_nuclei", midbox_mask = None)
    
    
        # generate chromosome objects to analyze
        if not _use_pre_defined_Z: # if use pre-specified Z for main chromosome
            main_chromosome_objects = simple_find_objects_timelapsed_3D (_main_chromosome_mask, combine_objects = False,
                                        search_radius = 75, object_area_change_ratio = 0.6,
                                        object_erosion_factor =3)
    
    
    
    # set df for all timepoints (check the df columns definition above for details)
    _measurement_df_all_fov = pd.DataFrame(columns=core_noncore_measurement_columns)
    
    #================================analyze each timepoint========================================#
     
    # _fov_id ~ _timepoint_index
    for _fov_id in range(len(sorted_mm_data_array)):
        
        
        print ('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
        print (f'-- Analyze TIMEPOINT {_fov_id + 1}')

        
        # set df for measurement (check the df columns definition above for details)
        _measurement_df_each_fov = pd.DataFrame()
        
        # set mask list to save
        _mask_each_fov = []
        
        # load image data for the corresponding timepoint
        ims_each_fov_3D = sorted_mm_data_array [_fov_id]
        
        
        # get pseudo or saved dna mask
        raw_image_shape = ims_each_fov_3D[0,0,:,:].shape
        saved_dna_mask = generate_masks_from_saved (mask_filename_list[_fov_id],raw_image_shape)
        
        saved_dna_mask = saved_dna_mask [int(cell_center[1]-cell_size/2):int(cell_center[1]+cell_size/2),  #crop x,y
                         int(cell_center[0]-cell_size/2):int(cell_center[0]+cell_size/2)]  
        
          
        # crop raw images if necessary
        if threshold_by_region_method == "USER":
            ims_each_fov_3D = ims_each_fov_3D[:,:,
                        int(cell_center[1]-cell_size/2):int(cell_center[1]+cell_size/2),    # channel crop x
                         int(cell_center[0]-cell_size/2):int(cell_center[0]+cell_size/2)]   # channel crop y
            
        ############################################################
        # Analyze the main chromosome masses:
        print ('-- Analyze the main chromosome masses.')
        
        # if use pre-specified Z for main chromosome
        if _use_pre_defined_Z: 
            # load xy for all channels for the pre-specified Z
            ims_ch_xy = ims_each_fov_3D [:,z_selected-1,:,:]
            object_z_used = z_selected
            
            # generate main chromosome mask 
            if not inverse_dna: 
                dna_mask = chromosome_mask_segmentation(ims_ch_xy,
                                                    dna_ch_index=new_dna_ch_index, 
                                                             cell_mask_ch_index =new_myo_ch_index, cell_center=cell_center,
                                                             cell_size=cell_size, 
                                                    small_object_th=large_object_size,
                                         threshold_by_region = "GLOBAL",inner_core_mode = False, 
                                                    chr_mask_from_saved = saved_dna_mask)
            else:            
                dna_mask = inverse_chromosome_mask_segmentation_by_line_profile(ims_ch_xy,
                                                    dna_ch_index=new_dna_ch_index, 
                                                             cell_center=cell_center,
                                                             cell_size=cell_size, 
                                                            extra_erosion_factor = extra_erosion_factor,
                                         threshold_by_region = "GLOBAL",inner_core_mode = False, 
                                                non_chr_low_ratio =0.09, chr_peak_prominence = 10,
                                                    chr_mask_from_saved = saved_dna_mask)
                
            # repeat the size filtering once with the same setting in case of unknown error occured in prior function
            dna_mask= dna_mask.astype(bool)
            dna_mask = remove_small_objects(dna_mask, large_object_size)
            
            
            labeled_main_chromosome, num_of_main_chromosome = ndi.label(dna_mask)
            dna_mask_to_exclude=erosion(dna_mask, disk(10))
                            
            if num_of_main_chromosome == 2:
                
                
                # this generate or replaced the midbox based on the refined dna mask 
                refined_midbox_dict = generate_midbox_with_bins (dna_mask, num_of_bins=num_of_segments,_verbose = True)
            
                # get the midbox for the core by excluding the midbox seg on the two edges
                midbox_for_core = np.zeros(dna_mask.shape)
                total_segs = len(refined_midbox_dict.values())
                for _seg_index, _seg in enumerate(refined_midbox_dict.values()):
                    if _seg_index>0 and _seg_index<total_segs-1:
                        midbox_for_core += _seg
                
                
                
                # Get masks for other channels
                # er mask for the cell
                if analyze_er:
                    _all_er_mask = er_membrane_mask_segmentation (ims_ch_xy,
                                                     er_ch_index =new_er_ch_index, 
                                                     cell_center=cell_center, cell_size=cell_size,
                                             threshold_by_region = 'GLOBAL',ridge_sigma=ridge_sigma,
                                                         sm_fragment_size =sm_fragment_size,
                                                             non_ridge_segmentation=non_ridge_segmentation,
                                                             enriched_membrane=enriched_membrane) 
                else:
                    print('No ER analysis, return empty values.')
                    _all_er_mask = np.zeros(ims_ch_xy[0].shape)
                    
                
                if analyze_myo:
                    if _actual_myo_ch == 'NUP' or _actual_myo_ch == 'MYO' or _actual_myo_ch == 'NE':
                        _all_nup_mask = myo_enrichment_mask_segmentation (ims_ch_xy,
                                                         myo_ch_index =new_myo_ch_index,
                                                         dna_mask = dna_mask_to_exclude,
                                                 cell_center=cell_center, cell_size=cell_size,
                                                         threshold_by_region = 'GLOBAL',
                                                num_of_obj = 2, std_ratio =std_ratio, 
                                                                 DNA_exclusion = DNA_exclusion_for_NUP)
                    
                    elif _actual_myo_ch == 'ACTIN' or _actual_myo_ch == 'ER':
                        _all_nup_mask = er_membrane_mask_segmentation (ims_ch_xy,
                                                     er_ch_index =new_myo_ch_index, 
                                                     cell_center=cell_center, cell_size=cell_size,
                                                             sm_fragment_size = actin_fragment_size,
                                                                   ridge_sigma=actin_ridge_sigma,
                                             threshold_by_region = 'GLOBAL') 
                    
                else:
                    print('No Nup (or Myosin) analysis, return empty values.')
                    _all_nup_mask = np.zeros(ims_ch_xy[0].shape)
                
   #########################################################################################################  

                mask_both_chr = {1:{},2:{}}
    
                # Start measuring for each subregions for channels of interest
                for _object_id, _main_chromosome_id in enumerate(range(num_of_main_chromosome)):
                    object_mask = (labeled_main_chromosome == _main_chromosome_id+1)
                    
                    # dict for mreasure df for each subregion
                    all_measurements_chr_dict={'inner_core':{},'outer_core':{},'noncore':{},'combined':{}}
                    
                    mask_each_chr = {'dna':[object_mask],'dna_rim':[],'er':[],'nup':[]}
                    mask_both_chr[_object_id+1]=mask_each_chr
                    
                    # add er measurement to the corresponding subregion
                    if analyze_er:
                        er_subregions = generate_core_noncore_for_chromosome(object_mask, midbox_for_core, 
                                                                         core_thickness=er_core_thickness)
                        combined_er_area =[]
                        combined_er_dna_rim=[]
                        
                        for _reg_label, _reg_mask in er_subregions.items():
                            er_enrich = _reg_mask*_all_er_mask
                            normalized_er_area = np.sum(er_enrich)/np.sum(_reg_mask)
                            # Also append the measurement for the integrated total inner core:
                            combined_er_area.append(np.sum(er_enrich))
                            combined_er_dna_rim.append(np.sum(_reg_mask))
                            
                            all_measurements_chr_dict[_reg_label]['er_recruit_normalized_area'] =normalized_er_area
                            all_measurements_chr_dict[_reg_label]['er_recruit_area'] =np.sum(er_enrich)
                            all_measurements_chr_dict[_reg_label]['er_dna_rim'] =np.sum(_reg_mask)
                            all_measurements_chr_dict[_reg_label]['core_noncore_type'] =_reg_label
                        
                            mask_each_chr['er'].append(er_enrich)
                            mask_each_chr['dna_rim'].append(_reg_mask)
                            
                    # add myo/nup measurement to the corresponding subregion    
                    if analyze_myo:
                        nup_subregions = generate_core_noncore_for_chromosome(object_mask, midbox_for_core,
                                                                        chr_rim_adjust_size = nup_rim_adjust_size,
                                                                         core_thickness=nup_core_thickness)
                        
                        combined_nup_area =[]
                        combined_nup_dna_rim=[]
                    
                        for _reg_label, _reg_mask in nup_subregions.items():
                            nup_enrich = _reg_mask*_all_nup_mask
                            normalized_nup_area = np.sum(nup_enrich)/np.sum(_reg_mask)
                            # Also append the measurement for the integrated total inner core:
                            combined_nup_area.append(np.sum(nup_enrich))
                            combined_nup_dna_rim.append(np.sum(_reg_mask))
                            
                            all_measurements_chr_dict[_reg_label][f'{myo_df_prefix}_recruit_normalized_area'] =normalized_nup_area
                            all_measurements_chr_dict[_reg_label][f'{myo_df_prefix}_recruit_area'] =np.sum(nup_enrich)
                            all_measurements_chr_dict[_reg_label][f'{myo_df_prefix}_dna_rim'] =np.sum(_reg_mask)
                            all_measurements_chr_dict[_reg_label]['core_noncore_type'] =_reg_label
                        
                            mask_each_chr['nup'].append(nup_enrich)
                                       
                    # add the combined all area measurements for the whole chromosome       
                    if analyze_er:
                        er_recruit_area= sum(combined_er_area)
                        er_dna_rim = sum(combined_er_dna_rim)
                        normalized_er_area = er_recruit_area/ er_dna_rim 
                    else:
                        er_recruit_area= np.nan
                        er_dna_rim = np.nan
                        normalized_er_area = np.nan
                        
            
                    if analyze_myo:
                        nup_recruit_area= sum(combined_nup_area)    
                        nup_dna_rim = sum(combined_nup_dna_rim)
                        normalized_nup_area = nup_recruit_area/nup_dna_rim
                    else:
                        nup_recruit_area= np.nan
                        nup_dna_rim = np.nan
                        normalized_nup_area = np.nan
                                        
                    all_measurements_chr_dict['combined']['er_recruit_normalized_area'] =normalized_er_area
                    all_measurements_chr_dict['combined'][f'{myo_df_prefix}_recruit_normalized_area'] =normalized_nup_area
                    all_measurements_chr_dict['combined']['er_recruit_area'] =er_recruit_area
                    all_measurements_chr_dict['combined'][f'{myo_df_prefix}_recruit_area'] =nup_recruit_area
                    all_measurements_chr_dict['combined']['er_dna_rim'] =er_dna_rim
                    all_measurements_chr_dict['combined'][f'{myo_df_prefix}_dna_rim'] =nup_dna_rim
                    all_measurements_chr_dict['combined']['core_noncore_type'] = 'whole_chromosome'
                    
                    # concate the subregion and whole chromosome for each chromosome
                    all_measurements_each_chr = pd.DataFrame(columns=core_noncore_measurement_columns[5:])
                    all_measurements_each_chr=pd.concat([all_measurements_each_chr, 
                                            pd.DataFrame(all_measurements_chr_dict.values())], ignore_index=True, axis=0) 
                    
                    all_measurements_each_chr['object_id'] = _object_id +1
                    all_measurements_each_chr['object_z_slice'] =object_z_used
                    all_measurements_each_chr['object_type'] ='main_chromosome'
                                    
                    # append each concatanted chr to each fov
                    _measurement_df_each_fov = pd.concat([_measurement_df_each_fov, all_measurements_each_chr])   
                # add info for each fov
                _measurement_df_each_fov.insert(0, 'cell_id', cell_id)
                _measurement_df_each_fov.insert(1, 'time_interval',time_interval)
                _measurement_df_each_fov.insert(1, 'timepoint', _fov_id + 1)
                _measurement_df_each_fov.insert(0, 'data_path', data_saved_folder.split("\\")[-1])
                
                # add each fov to all fov    
                _measurement_df_all_fov = pd.concat([_measurement_df_all_fov, _measurement_df_each_fov]) 
                print (f'Analysis for TIMEPOINT_{_fov_id+1} is complete.')  
                
          #####################################################################      
                # Individual masks to save for each fov:
                mask_save_path =data_analysis_folder +os.sep+"core_noncore_segmentations"+os.sep+cell_id
                if not os.path.exists(mask_save_path):
                    os.makedirs(mask_save_path)
                mask_savename = [] 
                if timelapsed == True:
                    mask_name_prefix = 'time_'
                else:
                    mask_name_prefix = 'fov_'
                
                # save shared mask for both chromosomes
                global_masks = [_all_er_mask,_all_nup_mask]
                global_mask_savenames = []
                global_mask_savenames.append(mask_name_prefix + str(_fov_id+1) + "_er_mask.tif")
                global_mask_savenames.append(mask_name_prefix + str(_fov_id+1) + f"_{myo_df_prefix}_mask.tif")
                
                for _mask, _mask_names in zip(global_masks,global_mask_savenames):
                    io.imsave(mask_save_path + os.sep + _mask_names, skimage.img_as_ubyte(_mask), check_contrast=False)
                 
                # save masks for each chromosome
                for _chr_id, _masks_dict in mask_both_chr.items():
                    _dna_mask_name = mask_name_prefix + str(_fov_id+1) + f"_main_chr_{_chr_id}_dna.tif"
                    io.imsave(mask_save_path + os.sep + _dna_mask_name, skimage.img_as_ubyte(_masks_dict['dna']), check_contrast=False)
                    # save mask for er
                    if analyze_er:
                        _er_mask_name = mask_name_prefix + str(_fov_id+1) + f"_main_chr_{_chr_id}_er.tif"
                        er_recruit_mask_combined = np.zeros(_all_er_mask.shape)
                        for _reg_ind, _mask in enumerate(_masks_dict['er']):
                            # subregion index start from 0
                            _mask = _mask * (_reg_ind+1)
                            er_recruit_mask_combined += _mask   
                        er_recruit_mask_combined = color.label2rgb(er_recruit_mask_combined, bg_label=0)
                        io.imsave(mask_save_path + os.sep + _er_mask_name, skimage.img_as_ubyte(er_recruit_mask_combined), check_contrast=False)
                        # save defualt dna rim mask using er core thickness
                        _dna_rim_mask_name = mask_name_prefix + str(_fov_id+1) + f"_main_chr_{_chr_id}_rim_default.tif"
                        dna_rim_mask_combined = np.zeros(_all_er_mask.shape)
                        for _reg_ind, _mask in enumerate(_masks_dict['dna_rim']):
                            _mask = _mask * (_reg_ind+1)
                            dna_rim_mask_combined+=_mask
                        dna_rim_mask_combined = color.label2rgb(dna_rim_mask_combined, bg_label=0)
                        io.imsave(mask_save_path + os.sep + _dna_rim_mask_name, skimage.img_as_ubyte(dna_rim_mask_combined), check_contrast=False)
                  
                    # save mask for myo/nup
                    if analyze_myo:
                        _nup_mask_name = mask_name_prefix + str(_fov_id+1) + f"_main_chr_{_chr_id}_{myo_df_prefix}.tif"
                        nup_recruit_mask_combined = np.zeros(_all_nup_mask.shape)
                        for _reg_ind, _mask in enumerate(_masks_dict['nup']):
                            # subregion index start from 0
                            _mask = _mask * (_reg_ind+1)
                            nup_recruit_mask_combined += _mask   
                        nup_recruit_mask_combined = color.label2rgb(nup_recruit_mask_combined, bg_label=0)
                        io.imsave(mask_save_path + os.sep + _nup_mask_name, skimage.img_as_ubyte(nup_recruit_mask_combined), check_contrast=False)
            ############################################################    
                                
#============================= save results as excel for all timepoints ==========================================#
   # Export measurements to excel for each cell
    # reset the index           
    #_measurement_df_all_fov.reset_index(drop=True, inplace=True)

    analysis_save_path =data_analysis_folder +os.sep+"measurements"
    if not os.path.exists(analysis_save_path):
        os.makedirs(analysis_save_path)
        
    _whole_chr_df = _measurement_df_all_fov[_measurement_df_all_fov['core_noncore_type']=='whole_chromosome']
    _sub_chr_df = _measurement_df_all_fov[_measurement_df_all_fov['core_noncore_type'].str.contains('whole_chromosome')==False]
    
    _whole_chr_analysis_savename = f'measurement_for_{cell_id}_whole_chromosome.xlsx'
    _whole_chr_df.to_excel(analysis_save_path+os.sep+_whole_chr_analysis_savename,index=False)
         
    _sub_chr_analysis_savename = f'measurement_for_{cell_id}_core_noncore.xlsx'
    _sub_chr_df.to_excel(analysis_save_path+os.sep+_sub_chr_analysis_savename,index=False)
    
    print(f"Measurement for cell {cell_id} saved.")
    print("Analysis completed in %.2f seconds." % (time.time() - start_time))  
    print ('-----------------------------------------------------------------------------------------------------')
    print ('-----------------------------------------------------------------------------------------------------')
              
           

    

Starting analysis for cell_cell1
-----------------------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------------------
There are 4 fovs/timepoints for this 4-channel, 4-slice dataset.
Number of masks provided do not match number of fovs/timepoints. Use saved masks for timepoints that have match only.
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
-- Analyze TIMEPOINT 1
No saved mask. Use the default empty mask.
-- Analyze the main chromosome masses.
Segmentation of chromosome using otsu (or other defualt) method.
Midboxes dict with 5 bins are generated.
There are 441 small membrane fragments (sigma = 2) removed after converting detected ridges to binary images.
Using 157.43 to segment the myo enriched region.
Core-Noncore segmenations are generated for the input chromosome mask.
Core-Noncore segmenations are generated for the input ch

C:\Users\Shiwei\Documents\AnalysisTool\mitosis_analysis.py:327: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if mask_file [-3:] == 'roi':
C:\Users\Shiwei\Documents\AnalysisTool\mitosis_analysis.py:338: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if mask_file [-3:] == 'tif':


Segmentation of chromosome using otsu (or other defualt) method.
Midboxes dict with 5 bins are generated.
There are 641 small membrane fragments (sigma = 2) removed after converting detected ridges to binary images.
Using 143.47 to segment the myo enriched region.
Core-Noncore segmenations are generated for the input chromosome mask.
Core-Noncore segmenations are generated for the input chromosome mask.
Core-Noncore segmenations are generated for the input chromosome mask.
Core-Noncore segmenations are generated for the input chromosome mask.
Analysis for TIMEPOINT_1 is complete.
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
-- Analyze TIMEPOINT 2
No saved mask. Use the default empty mask.
-- Analyze the main chromosome masses.
Segmentation of chromosome using otsu (or other defualt) method.
Midboxes dict with 5 bins are generated.
There are 630 small membrane fragments (sigma = 2) removed after converting detected ridges to binary images.
Using 139

C:\Users\Shiwei\Documents\AnalysisTool\mitosis_analysis.py:327: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if mask_file [-3:] == 'roi':
C:\Users\Shiwei\Documents\AnalysisTool\mitosis_analysis.py:338: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if mask_file [-3:] == 'tif':


Midboxes dict with 5 bins are generated.
There are 41 small membrane fragments (sigma = 2) removed after converting detected ridges to binary images.
Using 138.46 to segment the myo enriched region.
Core-Noncore segmenations are generated for the input chromosome mask.
Core-Noncore segmenations are generated for the input chromosome mask.
Core-Noncore segmenations are generated for the input chromosome mask.
Core-Noncore segmenations are generated for the input chromosome mask.
Analysis for TIMEPOINT_1 is complete.
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
-- Analyze TIMEPOINT 2
No saved mask. Use the default empty mask.
-- Analyze the main chromosome masses.
Segmentation of chromosome using otsu (or other defualt) method.
Midboxes dict with 5 bins are generated.
There are 67 small membrane fragments (sigma = 2) removed after converting detected ridges to binary images.
Using 138.21 to segment the myo enriched region.
Core-Noncore segmenations a

C:\Users\Shiwei\Documents\AnalysisTool\mitosis_analysis.py:327: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if mask_file [-3:] == 'roi':
C:\Users\Shiwei\Documents\AnalysisTool\mitosis_analysis.py:338: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if mask_file [-3:] == 'tif':


There are 355 small membrane fragments (sigma = 2) removed after converting detected ridges to binary images.
Using 155.79 to segment the myo enriched region.
Core-Noncore segmenations are generated for the input chromosome mask.
Core-Noncore segmenations are generated for the input chromosome mask.
Core-Noncore segmenations are generated for the input chromosome mask.
Core-Noncore segmenations are generated for the input chromosome mask.
Analysis for TIMEPOINT_1 is complete.
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
-- Analyze TIMEPOINT 2
No saved mask. Use the default empty mask.
-- Analyze the main chromosome masses.
Segmentation of chromosome using otsu (or other defualt) method.
Midboxes dict with 5 bins are generated.
There are 416 small membrane fragments (sigma = 2) removed after converting detected ridges to binary images.
Using 153.16 to segment the myo enriched region.
Core-Noncore segmenations are generated for the input chromosome m

# 6. Additionally save the current notebook in the data folder (optional) 

In [34]:
# Save extra copy of the notebook in the data folder (optionally)
import ipynbname
import shutil

# get current jupyter notebook file and path
nb_path = ipynbname.path()
print(f'Getting the notebook filename.')
ori_fielname = str(nb_path)
#ori_fielname

target_fielname = data_saved_folder+os.sep+ori_fielname.split('\\')[-1]
target_fielname

Getting the notebook filename.


'D:\\Analyzed_CellBio\\Jing\\3HeLa_NPC_assembly_on_laggard\\20211103 HeLa GFP-Nup107 mCherry-STIM1 sirDNA Y27 plus MPS1i exp2\\exp1 1uM MPS1i\\20211103_HeLa_Nup107_STIM1_dna_MPS1-Y27_exp1.ipynb'

In [35]:
check_savetime = True
time_diff=45
while check_savetime:
    current_t = time.time()
    nb_time = os.path.getmtime(ori_fielname)
    if round(current_t-nb_time)<=time_diff:
        print(f'Notebook has been saved under {time_diff}s ago.')
        check_savetime=False
                
# save (and overwrite) the notebook also in the data folder

if not check_savetime:
    if os.path.exists(target_fielname):
        os.remove(target_fielname)
    print('Copy notebook to the datafolder')
    shutil.copy2(ori_fielname, target_fielname)


Notebook has been saved under 45s ago.
Copy notebook to the datafolder
